# Symbolic Regression GP (CUDA Enabled)
## Instructions
1. Go to **Runtime -> Change runtime type** and select **T4 GPU** (or any available GPU).
2. Run all cells to compile and execute the project.


In [ ]:
!nvidia-smi

In [ ]:
import os
os.makedirs('src', exist_ok=True)
os.makedirs('build', exist_ok=True)

In [ ]:
import os

files_to_create = {
  "CMakeLists.txt": "cmake_minimum_required(VERSION 3.18)\n\n# Initially only enable CXX. We will enable CUDA if found.\nproject(SymbolicRegressionGP LANGUAGES CXX)\n\n# Fix for \"PTX was compiled with an unsupported toolchain\" on Colab T4\n# This forces generation of SASS (binary) for the current GPU, avoiding JIT issues.\nset(CMAKE_CUDA_ARCHITECTURES native)\n\nset(CMAKE_CXX_STANDARD 17)\nset(CMAKE_CXX_STANDARD_REQUIRED True)\n\n# Find OpenMP (Required for CPU parallelism)\nfind_package(OpenMP REQUIRED COMPONENTS CXX)\n\n# --- CUDA Detection ---\ninclude(CheckLanguage)\ncheck_language(CUDA)\n\nset(SOURCES\n    src/main.cpp\n    src/ExpressionTree.cpp\n    src/Fitness.cpp\n    src/GeneticOperators.cpp\n    src/AdvancedFeatures.cpp\n    src/GeneticAlgorithm.cpp\n)\n\nif(CMAKE_CUDA_COMPILER)\n    message(STATUS \"CUDA compiler found: ${CMAKE_CUDA_COMPILER}\")\n    enable_language(CUDA)\n    \n    # Suppress warning about FindCUDA being deprecated (CMP0146)\n    cmake_policy(SET CMP0146 OLD)\n    find_package(CUDA REQUIRED)\n\n    # Add .cu file to sources only if CUDA is present\n    list(APPEND SOURCES src/FitnessGPU.cu)\n\n    # Create Executable\n    add_executable(SymbolicRegressionGP ${SOURCES})\n\n    # Define the macro to enable GPU code paths in C++\n    target_compile_definitions(SymbolicRegressionGP PUBLIC \"USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\")\n\n    # Include CUDA dirs\n    target_include_directories(SymbolicRegressionGP PUBLIC ${CUDA_INCLUDE_DIRS})\n\n    # Link CUDA libraries\n    # Try modern target first, fallback to variables\n    if(TARGET CUDA::cudart)\n        target_link_libraries(SymbolicRegressionGP PUBLIC CUDA::cudart)\n    else()\n        target_link_libraries(SymbolicRegressionGP PUBLIC ${CUDA_LIBRARIES})\n    endif()\n\n    set(CMAKE_CUDA_PROPAGATE_HOST_FLAGS ON)\n    \n    message(STATUS \"Build type: GPU Accelerated\")\nelse()\n    message(STATUS \"CUDA compiler NOT found. Building for CPU only.\")\n    \n    # Create Executable (without .cu file)\n    add_executable(SymbolicRegressionGP ${SOURCES})\n    \n    message(STATUS \"Build type: CPU Only\")\nendif()\n\n# Common settings\ntarget_include_directories(SymbolicRegressionGP PUBLIC src)\n\n# Enlazar las librer\u00edas necesarias.\n# Se usa la variable legacy ${CUDA_LIBRARIES} como m\u00e9todo de compatibilidad\n# ya que el target moderno CUDA::cudart no se est\u00e1 encontrando en este sistema.\ntarget_link_libraries(SymbolicRegressionGP PUBLIC\n    # Enlazar con OpenMP para C++\n    OpenMP::OpenMP_CXX\n\n    # Enlazar con las librer\u00edas de CUDA (m\u00e9todo de compatibilidad)\n    ${CUDA_LIBRARIES}\n)\n\n# Asegura que los flags del compilador de C++ (como /std:c++17) se pasen a nvcc.\nset(CMAKE_CUDA_PROPAGATE_HOST_FLAGS ON)\n\n\n# === OPTIMIZACI\u00d3N: Flags de compilaci\u00f3n agresivos para GCC/Clang (Colab) ===\n# NOTA: Para MSVC+CUDA, dejamos que CMake maneje la optimizaci\u00f3n autom\u00e1ticamente\n#       ya que los flags manuales causan conflictos con nvcc\nif(CMAKE_CXX_COMPILER_ID MATCHES \"GNU|Clang\")\n    target_compile_options(SymbolicRegressionGP PRIVATE\n        $<$<COMPILE_LANGUAGE:CXX>:-O3>              # M\u00e1xima optimizaci\u00f3n\n        $<$<COMPILE_LANGUAGE:CXX>:-march=native>    # Optimizar para CPU actual\n        $<$<COMPILE_LANGUAGE:CXX>:-ffast-math>      # Matem\u00e1ticas r\u00e1pidas\n        $<$<COMPILE_LANGUAGE:CXX>:-funroll-loops>   # Desenrollar loops\n        $<$<COMPILE_LANGUAGE:CXX>:-ftree-vectorize> # Forzar vectorizaci\u00f3n\n    )\n    # Link-time optimization (LTO) para Release\n    if(CMAKE_BUILD_TYPE STREQUAL \"Release\")\n        set_property(TARGET SymbolicRegressionGP PROPERTY INTERPROCEDURAL_OPTIMIZATION TRUE)\n    endif()\nendif()\n",
  "src/AdvancedFeatures.cpp": "#include \"AdvancedFeatures.h\"\n#include \"Globals.h\"\n#include \"GeneticOperators.h\"\n#include \"Fitness.h\"\n#include \"ExpressionTree.h\" // Necesario para tree_to_string en la simplificaci\u00f3n\n#include <cmath>\n#include <numeric>\n#include <algorithm>\n#include <iostream>\n#include <unordered_map>\n#include <vector>\n\n//---------------------------------\n// EvolutionParameters\n//---------------------------------\nEvolutionParameters EvolutionParameters::create_default() {\n    // Usa constantes globales para los valores por defecto\n    return {BASE_MUTATION_RATE, BASE_ELITE_PERCENTAGE, DEFAULT_TOURNAMENT_SIZE, DEFAULT_CROSSOVER_RATE};\n}\n\nvoid EvolutionParameters::mutate(int stagnation_counter) {\n    auto& rng = get_rng();\n    double aggression_factor = 1.0;\n    // Ajuste del factor de agresi\u00f3n basado en el estancamiento\n    if (stagnation_counter > STAGNATION_LIMIT_ISLAND / 2) {\n        // Aumenta la agresi\u00f3n si hay estancamiento significativo\n        aggression_factor = 1.0 + (static_cast<double>(stagnation_counter - STAGNATION_LIMIT_ISLAND / 2) / (STAGNATION_LIMIT_ISLAND / 2.0)) * 0.5; // Escala de 1.0 a 1.5\n        aggression_factor = std::min(aggression_factor, 2.0); // Limitar la agresi\u00f3n m\u00e1xima\n    } else if (stagnation_counter < STAGNATION_LIMIT_ISLAND / 4 && stagnation_counter > 0) {\n        // Reduce la agresi\u00f3n si no hay mucho estancamiento, pero no es 0\n        aggression_factor = 1.0 - (static_cast<double>(STAGNATION_LIMIT_ISLAND / 4 - stagnation_counter) / (STAGNATION_LIMIT_ISLAND / 4.0)) * 0.5; // Escala de 0.5 a 1.0\n        aggression_factor = std::max(aggression_factor, 0.5); // Limitar la agresi\u00f3n m\u00ednima\n    } else if (stagnation_counter == 0) {\n        // Muy poco estancamiento, cambios muy peque\u00f1os\n        aggression_factor = 0.2; // Cambios muy conservadores\n    }\n\n    std::uniform_real_distribution<double> base_rate_change(-0.05, 0.05);\n    std::uniform_int_distribution<int> base_tourney_change(-2, 2);\n\n    double rate_change_val = base_rate_change(rng) * aggression_factor;\n    int tourney_change_val = static_cast<int>(std::round(base_tourney_change(rng) * aggression_factor));\n    \n    // Asegurar que haya alg\u00fan cambio si la agresi\u00f3n es alta y el cambio base es 0\n    if (aggression_factor > 1.0 && tourney_change_val == 0 && base_tourney_change(rng) != 0) {\n         tourney_change_val = (base_tourney_change(rng) > 0) ? 1 : -1;\n    }\n\n    // Definir l\u00edmites din\u00e1micos para los par\u00e1metros\n    double min_mutation = 0.05;\n    double max_mutation_base = 0.5;\n    double max_mutation = min_mutation + (max_mutation_base - min_mutation) * (1.0 + aggression_factor / 2.0);\n\n    double min_elite = 0.02;\n    double max_elite_base = 0.25;\n    double max_elite = min_elite + (max_elite_base - min_elite) * (1.0 + aggression_factor / 2.0);\n\n    int min_tournament = 3;\n    int max_tournament_base = 30;\n    int max_tournament = min_tournament + static_cast<int>((max_tournament_base - min_tournament) * (1.0 + aggression_factor / 2.0));\n\n    // Aplicar los cambios y asegurar que est\u00e9n dentro de los l\u00edmites\n    mutation_rate = std::clamp(mutation_rate + rate_change_val, min_mutation, max_mutation);\n    elite_percentage = std::clamp(elite_percentage + rate_change_val, min_elite, max_elite);\n    tournament_size = std::clamp(tournament_size + tourney_change_val, min_tournament, max_tournament);\n    crossover_rate = std::clamp(crossover_rate + rate_change_val, 0.5, 0.95);\n}\n\n//---------------------------------\n// PatternMemory\n//---------------------------------\nvoid PatternMemory::record_success(const NodePtr& tree, double fitness) {\n    std::string pattern = extract_pattern(tree);\n    if (pattern.empty() || pattern.length() > 50 || pattern.length() < 3 || pattern == \"N\") return;\n    auto it = patterns.find(pattern);\n    if (it == patterns.end()) {\n        patterns[pattern] = {pattern, fitness, 1, (fitness < INF ? 1.0 : 0.0)};\n    } else {\n        auto& p = it->second;\n        p.uses++;\n        double improvement = (fitness < p.best_fitness && p.best_fitness < INF) ? 1.0 : 0.0;\n        p.success_rate = ((p.success_rate * (p.uses - 1)) + improvement) / p.uses;\n        p.best_fitness = std::min(p.best_fitness, fitness);\n    }\n}\n\nNodePtr PatternMemory::suggest_pattern_based_tree(int max_depth) {\n    if (patterns.empty()) return nullptr;\n    std::vector<std::pair<std::string, double>> candidates;\n    for (const auto& [pattern_str, info] : patterns) {\n        if (info.uses >= PATTERN_MEM_MIN_USES && (info.success_rate > 0.1 || info.best_fitness < PATTERN_RECORD_FITNESS_THRESHOLD)) {\n             double weight = info.success_rate + (1.0 / (1.0 + info.best_fitness));\n             candidates.emplace_back(pattern_str, weight);\n        }\n    }\n    if (candidates.empty()) return nullptr;\n    std::vector<double> weights;\n    std::transform(candidates.begin(), candidates.end(), std::back_inserter(weights), [](const auto& p){ return p.second; });\n    std::discrete_distribution<> dist(weights.begin(), weights.end());\n    auto& rng = get_rng();\n    int selected_idx = dist(rng);\n    return parse_pattern(candidates[selected_idx].first, max_depth);\n}\n\nstd::string PatternMemory::extract_pattern(const NodePtr& node) {\n    if (!node) return \"N\";\n    switch (node->type) {\n        case NodeType::Constant: return \"#\";\n        case NodeType::Variable: return \"x\";\n        case NodeType::Operator:\n            return \"(\" + extract_pattern(node->left) + node->op + extract_pattern(node->right) + \")\";\n        default: return \"?\";\n    }\n}\n\nNodePtr PatternMemory::parse_pattern(const std::string& pattern, int max_depth) {\n    // Placeholder implementation\n    if (pattern == \"#\") {\n        auto node = std::make_shared<Node>(NodeType::Constant);\n        if (FORCE_INTEGER_CONSTANTS) { std::uniform_int_distribution<int> cd(CONSTANT_INT_MIN_VALUE, CONSTANT_INT_MAX_VALUE); node->value = static_cast<double>(cd(get_rng())); }\n        else { std::uniform_real_distribution<double> cd(CONSTANT_MIN_VALUE, CONSTANT_MAX_VALUE); node->value = cd(get_rng()); }\n        if(std::fabs(node->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) node->value = 0.0;\n        return node;\n    }\n    if (pattern == \"x\") return std::make_shared<Node>(NodeType::Variable);\n    if (pattern == \"N\") return nullptr;\n    if (pattern.length() > 3 && pattern.front() == '(' && pattern.back() == ')') {\n          return generate_random_tree(max_depth); // Fallback\n     }\n    return generate_random_tree(max_depth); // Fallback\n}\n\n//---------------------------------\n// Pareto Optimizer\n//---------------------------------\nParetoSolution::ParetoSolution(NodePtr t, double acc, double complexity_val) : tree(std::move(t)), accuracy(acc), complexity(complexity_val), dominated(false) {}\n\nbool ParetoSolution::dominates(const ParetoSolution& other) const {\n    bool better_in_one = (accuracy < other.accuracy) || (complexity < other.complexity);\n    bool not_worse_in_any = (accuracy <= other.accuracy) && (complexity <= other.complexity);\n    return better_in_one && not_worse_in_any;\n}\n\nvoid ParetoOptimizer::update(const std::vector<Individual>& population, const std::vector<double>& targets, const std::vector<double>& x_values) {\n    std::vector<ParetoSolution> candidates = pareto_front;\n    for (const auto& ind : population) {\n        if (ind.tree && ind.fitness_valid && ind.fitness < INF) {\n            candidates.emplace_back(ind.tree, ind.fitness, static_cast<double>(tree_size(ind.tree)));\n        }\n    }\n    for (auto& sol1 : candidates) {\n        sol1.dominated = false;\n        for (const auto& sol2 : candidates) {\n            if (&sol1 == &sol2) continue;\n            if (sol2.dominates(sol1)) { sol1.dominated = true; break; }\n        }\n    }\n    pareto_front.clear();\n    std::copy_if(candidates.begin(), candidates.end(), std::back_inserter(pareto_front),\n                 [](const auto& sol) { return !sol.dominated; });\n    if (pareto_front.size() > PARETO_MAX_FRONT_SIZE) {\n        std::sort(pareto_front.begin(), pareto_front.end(), [](const auto& a, const auto& b){ return a.accuracy < b.accuracy; });\n        pareto_front.resize(PARETO_MAX_FRONT_SIZE);\n    }\n}\n\nstd::vector<NodePtr> ParetoOptimizer::get_pareto_solutions() {\n    std::vector<NodePtr> result;\n    result.reserve(pareto_front.size());\n    std::transform(pareto_front.begin(), pareto_front.end(), std::back_inserter(result),\n                   [](const auto& sol) { return sol.tree; });\n    return result;\n}\n\n//---------------------------------\n// Domain Constraints\n//---------------------------------\nbool DomainConstraints::is_valid_recursive(const NodePtr& node) {\n     if (!node) return true;\n     if (node->type == NodeType::Operator) {\n         if (node->op == '/' && node->right && node->right->type == NodeType::Constant && std::fabs(node->right->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return false;\n         if (node->op == '^') { // Solo chequear 0^negativo/0\n              if (node->left && node->left->type == NodeType::Constant && std::fabs(node->left->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE &&\n                  node->right && node->right->type == NodeType::Constant && node->right->value <= SIMPLIFY_NEAR_ZERO_TOLERANCE) {\n                      return false;\n              }\n         }\n         if ((node->op == '*' || node->op == '/') && node->right && node->right->type == NodeType::Constant && std::fabs(node->right->value - 1.0) < SIMPLIFY_NEAR_ONE_TOLERANCE) return false;\n         if ((node->op == '+' || node->op == '-') && node->right && node->right->type == NodeType::Constant && std::fabs(node->right->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return false;\n         if (node->op == '*' && node->left && node->left->type == NodeType::Constant && std::fabs(node->left->value - 1.0) < SIMPLIFY_NEAR_ONE_TOLERANCE) return false;\n         if (node->op == '+' && node->left && node->left->type == NodeType::Constant && std::fabs(node->left->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return false;\n         if (!is_valid_recursive(node->left) || !is_valid_recursive(node->right)) return false;\n     }\n     return true;\n }\n\nbool DomainConstraints::is_valid(const NodePtr& tree) {\n    return is_valid_recursive(tree);\n}\n\nNodePtr DomainConstraints::simplify_recursive(NodePtr node) {\n    if (!node || node->type != NodeType::Operator) return node;\n    node->left = simplify_recursive(node->left);\n    node->right = simplify_recursive(node->right);\n\n    // Manejo de hijos nulos\n    if (node->left && !node->right) return node->left;\n    if (!node->left && node->right) return node->right;\n    if (!node->left && !node->right) { auto cn = std::make_shared<Node>(NodeType::Constant); cn->value = 1.0; return cn; }\n\n    // Constant Folding\n    if (node->left->type == NodeType::Constant && node->right->type == NodeType::Constant) {\n        try {\n            double result = evaluate_tree(node, 0.0);\n            if (!std::isnan(result) && !std::isinf(result)) {\n                auto cn = std::make_shared<Node>(NodeType::Constant);\n                if (FORCE_INTEGER_CONSTANTS) cn->value = std::round(result); else cn->value = result;\n                if (std::fabs(cn->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) cn->value = 0.0; return cn;\n            }\n        } catch (const std::exception&) {}\n    }\n    // Identity Simplifications & Fixes\n     if ((node->op == '+' || node->op == '-') && node->right->type == NodeType::Constant && std::fabs(node->right->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return node->left;\n     if (node->op == '+' && node->left->type == NodeType::Constant && std::fabs(node->left->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return node->right;\n     if ((node->op == '*' || node->op == '/') && node->right->type == NodeType::Constant && std::fabs(node->right->value - 1.0) < SIMPLIFY_NEAR_ONE_TOLERANCE) return node->left;\n     if (node->op == '*' && node->left && node->left->type == NodeType::Constant && std::fabs(node->left->value - 1.0) < SIMPLIFY_NEAR_ONE_TOLERANCE) return node->right;\n     if (node->op == '*' && ((node->left->type == NodeType::Constant && std::fabs(node->left->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) || (node->right->type == NodeType::Constant && std::fabs(node->right->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE))) { auto z = std::make_shared<Node>(NodeType::Constant); z->value = 0.0; return z; }\n     if (node->op == '^' && node->right->type == NodeType::Constant && std::fabs(node->right->value - 1.0) < SIMPLIFY_NEAR_ONE_TOLERANCE) return node->left; // A^1 -> A\n     if (node->op == '^' && node->right->type == NodeType::Constant && std::fabs(node->right->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) { auto o = std::make_shared<Node>(NodeType::Constant); o->value = 1.0; return o; } // A^0 -> 1\n    // Fix div by zero (constante)\n    if (node->op == '/' && node->right->type == NodeType::Constant && std::fabs(node->right->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) node->right->value = 1.0;\n\n    // --- NUEVAS REGLAS DE SIMPLIFICACI\u00d3N ---\n    // X / X = 1 (si X no es cero)\n    if (node->op == '/' && node->left && node->right) {\n        if (tree_to_string(node->left) == tree_to_string(node->right)) {\n            // Verificar que el divisor no sea cero para evitar 0/0\n            if (node->right->type != NodeType::Constant || std::fabs(node->right->value) >= SIMPLIFY_NEAR_ZERO_TOLERANCE) {\n                auto one = std::make_shared<Node>(NodeType::Constant);\n                one->value = 1.0;\n                return one;\n            }\n        }\n    }\n\n    // X - X = 0\n    if (node->op == '-' && node->left && node->right) {\n        if (tree_to_string(node->left) == tree_to_string(node->right)) {\n            auto zero = std::make_shared<Node>(NodeType::Constant);\n            zero->value = 0.0;\n            return zero;\n        }\n    }\n    // Ya no se hace clamp de exponente constante aqu\u00ed, se quit\u00f3 la restricci\u00f3n\n\n    return node;\n}\n\nNodePtr DomainConstraints::fix_or_simplify(NodePtr tree) {\n    if (!tree) return nullptr;\n    NodePtr cloned_tree = clone_tree(tree);\n    NodePtr simplified_tree = simplify_recursive(cloned_tree);\n    return simplified_tree;\n}\n\n//---------------------------------\n// Local Improvement\n//---------------------------------\n//---------------------------------\n// Local Improvement\n//---------------------------------\nvoid optimize_constants(NodePtr& tree, const std::vector<double>& targets, const std::vector<double>& x_values, double* d_targets, double* d_x_values) {\n    if (!tree) return;\n    \n    // 1. Collect constant nodes\n    std::vector<Node*> constants;\n    std::vector<Node*> stack;\n    stack.push_back(tree.get());\n    while(!stack.empty()){\n        Node* n = stack.back(); stack.pop_back();\n        if(!n) continue;\n        if(n->type == NodeType::Constant) constants.push_back(n);\n        else if(n->type == NodeType::Operator){\n            stack.push_back(n->right.get());\n            stack.push_back(n->left.get());\n        }\n    }\n    \n    if (constants.empty()) return;\n\n    // 2. Hill Climbing (Numeric Optimization)\n    int max_iter = 20; // Fast local search\n    auto& rng = get_rng();\n    \n    // Evaluate initial fitness\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n    double current_fitness = evaluate_fitness(tree, targets, x_values, d_targets, d_x_values);\n#else\n    double current_fitness = evaluate_fitness(tree, targets, x_values);\n#endif\n\n    std::normal_distribution<double> perturbation(0.0, 0.5); // Perturb standard deviation 0.5\n\n    for(int i=0; i<max_iter; ++i) {\n        // Select a random constant\n        int idx = std::uniform_int_distribution<int>(0, constants.size()-1)(rng);\n        double old_val = constants[idx]->value;\n        \n        // Perturb\n        double delta = perturbation(rng);\n        constants[idx]->value += delta;\n        if (FORCE_INTEGER_CONSTANTS) constants[idx]->value = std::round(constants[idx]->value);\n\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n        double new_fitness = evaluate_fitness(tree, targets, x_values, d_targets, d_x_values);\n#else\n        double new_fitness = evaluate_fitness(tree, targets, x_values);\n#endif\n\n        if (new_fitness < current_fitness) {\n            current_fitness = new_fitness; // Accept\n            // Adapt perturbation? Maybe reduce sigma?\n        } else {\n            constants[idx]->value = old_val; // Revert\n        }\n        \n        if (current_fitness < EXACT_SOLUTION_THRESHOLD) break;\n    }\n}\n\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\nstd::pair<NodePtr, double> try_local_improvement(const NodePtr& tree, double current_fitness, const std::vector<double>& targets, const std::vector<double>& x_values, int attempts, double* d_targets, double* d_x_values) {\n    // 1. First, try to optimize constants of the CURRENT tree\n    NodePtr optimized_tree = clone_tree(tree);\n    optimize_constants(optimized_tree, targets, x_values, d_targets, d_x_values);\n    double optimized_fitness = evaluate_fitness(optimized_tree, targets, x_values, d_targets, d_x_values);\n    \n    NodePtr best_neighbor = (optimized_fitness < current_fitness) ? optimized_tree : tree;\n    double best_neighbor_fitness = (optimized_fitness < current_fitness) ? optimized_fitness : current_fitness;\n\n    if (best_neighbor_fitness >= INF) return {best_neighbor, best_neighbor_fitness};\n\n    // 2. Structural Search (as before)\n    for (int i = 0; i < attempts; ++i) {\n        NodePtr neighbor = mutate_tree(best_neighbor, 1.0, 2); // Mutate the BEST so far\n        neighbor = DomainConstraints::fix_or_simplify(neighbor);\n        if (!neighbor) continue;\n        \n        // Also optimize constants of structural neighbor?\n        // Maybe too expensive. Let's do a quick random constant tweak.\n        // optimize_constants(neighbor, targets, x_values, d_targets, d_x_values); \n        \n        double neighbor_fitness = evaluate_fitness(neighbor, targets, x_values, d_targets, d_x_values);\n        if (neighbor_fitness < best_neighbor_fitness) {\n            best_neighbor = neighbor;\n            best_neighbor_fitness = neighbor_fitness;\n        }\n    }\n    return {best_neighbor, best_neighbor_fitness};\n}\n#else\nstd::pair<NodePtr, double> try_local_improvement(const NodePtr& tree, double current_fitness, const std::vector<double>& targets, const std::vector<double>& x_values, int attempts) {\n    // 1. First, try to optimize constants of the CURRENT tree\n    NodePtr optimized_tree = clone_tree(tree);\n    optimize_constants(optimized_tree, targets, x_values, nullptr, nullptr);\n    double optimized_fitness = evaluate_fitness(optimized_tree, targets, x_values);\n    \n    NodePtr best_neighbor = (optimized_fitness < current_fitness) ? optimized_tree : tree;\n    double best_neighbor_fitness = (optimized_fitness < current_fitness) ? optimized_fitness : current_fitness;\n\n    if (best_neighbor_fitness >= INF) return {best_neighbor, best_neighbor_fitness};\n\n    for (int i = 0; i < attempts; ++i) {\n        NodePtr neighbor = mutate_tree(best_neighbor, 1.0, 2);\n        neighbor = DomainConstraints::fix_or_simplify(neighbor);\n        if (!neighbor) continue;\n        double neighbor_fitness = evaluate_fitness(neighbor, targets, x_values);\n        if (neighbor_fitness < best_neighbor_fitness) {\n            best_neighbor = neighbor;\n            best_neighbor_fitness = neighbor_fitness;\n        }\n    }\n    return {best_neighbor, best_neighbor_fitness};\n}\n#endif\n\n//---------------------------------\n// Target Pattern Detection\n//---------------------------------\nstd::pair<std::string, double> detect_target_pattern(const std::vector<double>& targets) {\n    if (targets.size() < 3) return {\"none\", 0.0};\n    bool is_arithmetic = true; double diff = targets[1] - targets[0];\n    for (size_t i = 2; i < targets.size(); ++i) if (std::fabs((targets[i] - targets[i-1]) - diff) > 1e-6) { is_arithmetic = false; break; }\n    if (is_arithmetic) return {\"arithmetic\", diff};\n    bool is_geometric = true;\n    if (std::fabs(targets[0]) < 1e-9) {\n        bool all_zero = true; for(double t : targets) if (std::fabs(t) > 1e-9) { all_zero = false; break; }\n        if(all_zero) return {\"constant_zero\", 0.0}; else is_geometric = false;\n    }\n    if (is_geometric && std::fabs(targets[0]) >= 1e-9) {\n        double ratio = targets[1] / targets[0];\n        for (size_t i = 2; i < targets.size(); ++i) {\n             if (std::fabs(targets[i-1]) < 1e-9) { if (std::fabs(targets[i]) > 1e-9) { is_geometric = false; break; } }\n             else { if (std::fabs((targets[i] / targets[i-1]) - ratio) > 1e-6) { is_geometric = false; break; } }\n        }\n        if (is_geometric) return {\"geometric\", ratio};\n    }\n    return {\"none\", 0.0};\n}\n\n//---------------------------------\n// Generate Pattern Based Tree\n//---------------------------------\nNodePtr generate_pattern_based_tree(const std::string& pattern_type, double pattern_value) {\n    if (X_VALUES.empty() || RAW_TARGETS.empty()) return nullptr;\n    double a = RAW_TARGETS[0]; double x0 = X_VALUES[0];\n    if (pattern_type == \"arithmetic\") {\n        double d = pattern_value; auto root = std::make_shared<Node>(NodeType::Operator); root->op = '+';\n        auto cp = std::make_shared<Node>(NodeType::Constant); double cv = a - d * x0; if (FORCE_INTEGER_CONSTANTS) cv = std::round(cv); cp->value = (std::fabs(cv) < SIMPLIFY_NEAR_ZERO_TOLERANCE) ? 0.0 : cv; // Use RAW_TARGETS to avoid \"TARGETS\" not found\n\n        auto vp = std::make_shared<Node>(NodeType::Operator); vp->op = '*';\n        auto dc = std::make_shared<Node>(NodeType::Constant); double dv = d; if (FORCE_INTEGER_CONSTANTS) dv = std::round(dv); dc->value = (std::fabs(dv) < SIMPLIFY_NEAR_ZERO_TOLERANCE) ? 0.0 : dv;\n        auto xv = std::make_shared<Node>(NodeType::Variable); vp->left = dc; vp->right = xv;\n        root->left = cp; root->right = vp; return DomainConstraints::fix_or_simplify(root);\n    } else if (pattern_type == \"geometric\") {\n        double r = pattern_value; if (std::fabs(r) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return nullptr;\n        auto root = std::make_shared<Node>(NodeType::Operator); root->op = '*';\n        auto cp = std::make_shared<Node>(NodeType::Constant); double rpx0 = std::pow(r, x0); if (std::fabs(rpx0) < 1e-100) return nullptr;\n        double cv = a / rpx0; if (FORCE_INTEGER_CONSTANTS) cv = std::round(cv); cp->value = (std::fabs(cv) < SIMPLIFY_NEAR_ZERO_TOLERANCE) ? 0.0 : cv;\n        auto vp = std::make_shared<Node>(NodeType::Operator); vp->op = '^';\n        auto rc = std::make_shared<Node>(NodeType::Constant); double rv = r; if (FORCE_INTEGER_CONSTANTS) rv = std::round(rv); rc->value = (std::fabs(rv) < SIMPLIFY_NEAR_ZERO_TOLERANCE) ? 0.0 : rv;\n        auto xv = std::make_shared<Node>(NodeType::Variable); vp->left = rc; vp->right = xv;\n        root->left = cp; root->right = vp; return DomainConstraints::fix_or_simplify(root);\n    } else if (pattern_type == \"constant_zero\") {\n         auto node = std::make_shared<Node>(NodeType::Constant); node->value = 0.0; return node;\n     }\n    return nullptr; // No pattern tree generated\n}\n",
  "src/AdvancedFeatures.h": "#ifndef ADVANCEDFEATURES_H\n#define ADVANCEDFEATURES_H\n\n#include \"ExpressionTree.h\"\n#include \"Globals.h\" // Incluir Globals.h para INF\n#include <vector>\n#include <string>\n#include <map>\n#include <set>\n#include <utility> // Para std::pair\n#include <unordered_map>\n\n// Meta-evoluci\u00f3n: Par\u00e1metros que pueden adaptarse durante la ejecuci\u00f3n.\nstruct EvolutionParameters {\n    double mutation_rate;    // Tasa de mutaci\u00f3n actual\n    double elite_percentage; // Porcentaje de \u00e9lite actual\n    int tournament_size;     // Tama\u00f1o del torneo actual\n    double crossover_rate;   // Tasa de cruce actual\n\n    // Crea un conjunto de par\u00e1metros con valores por defecto (iniciales).\n    static EvolutionParameters create_default();\n\n    // Adapta (muta) los par\u00e1metros ligeramente.\n    // AHORA RECIBE el contador de estancamiento para ajustar la intensidad.\n    void mutate(int stagnation_counter);\n};\n\n// Memoria de patrones: Almacena sub-estructuras exitosas (Reinforcement Learning).\nclass PatternMemory {\n    struct PatternInfo {\n        std::string pattern_str; // Representaci\u00f3n del patr\u00f3n\n        double best_fitness = INF; // Mejor fitness visto para este patr\u00f3n\n        int uses = 0;             // N\u00famero de veces usado/visto\n        double success_rate = 0.0; // Tasa de \u00e9xito estimada\n    };\n    std::unordered_map<std::string, PatternInfo> patterns; // Mapa para almacenar patrones\n    int min_uses_for_suggestion = 3; // M\u00ednimo de usos para considerar sugerir un patr\u00f3n\n\npublic:\n    // Registra el \u00e9xito de un \u00e1rbol (y su patr\u00f3n) basado en su fitness.\n    void record_success(const NodePtr& tree, double fitness);\n    // Sugiere un \u00e1rbol basado en los patrones exitosos almacenados.\n    NodePtr suggest_pattern_based_tree(int max_depth);\n\nprivate:\n    // Extrae la representaci\u00f3n estructural (string) de un \u00e1rbol.\n    std::string extract_pattern(const NodePtr& tree);\n    // Intenta construir un \u00e1rbol a partir de un patr\u00f3n (string) - funci\u00f3n simplificada.\n    NodePtr parse_pattern(const std::string& pattern, int max_depth);\n};\n\n\n// Optimizaci\u00f3n Pareto: Mantiene un frente de soluciones no dominadas (compromiso precisi\u00f3n/complejidad).\nstruct ParetoSolution {\n    NodePtr tree = nullptr;   // \u00c1rbol de la soluci\u00f3n\n    double accuracy = INF;    // Objetivo 1: Precisi\u00f3n (fitness)\n    double complexity = INF;  // Objetivo 2: Complejidad (tama\u00f1o)\n    bool dominated = false;   // Bandera: \u00bfest\u00e1 dominada por otra soluci\u00f3n?\n\n    // Constructor por defecto (necesario si se usa en contenedores)\n    ParetoSolution() = default;\n    // Constructor principal\n    ParetoSolution(NodePtr t, double acc, double complexity_val);\n\n    // Comprueba si esta soluci\u00f3n domina a otra.\n    bool dominates(const ParetoSolution& other) const;\n};\n\nclass ParetoOptimizer {\n    std::vector<ParetoSolution> pareto_front; // Almacena las soluciones del frente\n    size_t max_front_size = 50; // L\u00edmite opcional para el tama\u00f1o del frente\n\npublic:\n    // Actualiza el frente de Pareto con individuos de la poblaci\u00f3n actual.\n    void update(const std::vector<struct Individual>& population, // Usa Individual struct\n                const std::vector<double>& targets,\n                const std::vector<double>& x_values);\n\n    // Obtiene los \u00e1rboles (NodePtr) de las soluciones en el frente actual.\n    std::vector<NodePtr> get_pareto_solutions();\n\n    // Obtiene una referencia constante al frente de Pareto completo.\n    const std::vector<ParetoSolution>& get_pareto_front() const { return pareto_front; }\n};\n\n\n// Restricciones de Dominio: Verifica y corrige/simplifica \u00e1rboles problem\u00e1ticos.\nclass DomainConstraints {\npublic:\n    // Comprueba si un \u00e1rbol cumple reglas b\u00e1sicas de validez est\u00e1tica.\n    static bool is_valid(const NodePtr& tree);\n\n    // Intenta simplificar/corregir un \u00e1rbol (devuelve una copia modificada).\n    static NodePtr fix_or_simplify(NodePtr tree);\n\nprivate:\n     // Ayudante recursivo para la simplificaci\u00f3n.\n    static NodePtr simplify_recursive(NodePtr node);\n    // Ayudante recursivo para la validaci\u00f3n est\u00e1tica.\n    static bool is_valid_recursive(const NodePtr& node);\n};\n\n// B\u00fasqueda Local: Intenta mejorar una soluci\u00f3n dada explorando vecinos cercanos.\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\nstd::pair<NodePtr, double> try_local_improvement(const NodePtr& tree,\n                                                  double current_fitness,\n                                                  const std::vector<double>& targets,\n                                                  const std::vector<double>& x_values,\n                                                  int attempts,\n                                                  double* d_targets, double* d_x_values);\n#else\nstd::pair<NodePtr, double> try_local_improvement(const NodePtr& tree,\n                                                  double current_fitness,\n                                                  const std::vector<double>& targets,\n                                                  const std::vector<double>& x_values,\n                                                  int attempts = 10);\n#endif\n\n\n// Detecci\u00f3n de Patrones en los Datos Objetivo.\nstd::pair<std::string, double> detect_target_pattern(const std::vector<double>& targets);\nNodePtr generate_pattern_based_tree(const std::string& pattern_type, double pattern_value);\n\n\n#endif // ADVANCEDFEATURES_H\n",
  "src/ExpressionTree.cpp": "#include \"ExpressionTree.h\"\n#include \"Globals.h\"\n#include <cmath>\n#include <limits>\n#include <stdexcept>\n#include <vector>\n#include <iostream>\n#include <iomanip>\n#include <string>\n#include <sstream>\n#include <stack>\n#include <unordered_map>\n#include <algorithm> // Para std::remove_if\n#include <cctype>    // Para isdigit, isspace\n#include <thread>    // Para thread_local RNG\n\n// --- Funci\u00f3n auxiliar para formatear constantes ---\n// --- Funci\u00f3n auxiliar para formatear constantes ---\nstd::string format_constant(double val) {\n    // Si es un entero o muy cercano a un entero, formatarlo como tal.\n    if (std::fabs(val - std::round(val)) < SIMPLIFY_NEAR_ZERO_TOLERANCE) {\n        return std::to_string(static_cast<long long>(std::round(val)));\n    } else {\n        std::ostringstream oss;\n        // Usar notaci\u00f3n cient\u00edfica para valores muy grandes o muy peque\u00f1os,\n        // o notaci\u00f3n fija para el resto, con precisi\u00f3n adecuada.\n        // Esto evita cadenas muy largas o p\u00e9rdida de informaci\u00f3n.\n        if (std::fabs(val) >= 1e6 || std::fabs(val) <= 1e-6) { // Umbrales ajustables\n            oss << std::scientific << std::setprecision(8) << val;\n        } else {\n            oss << std::fixed << std::setprecision(8) << val;\n        }\n        \n        std::string s = oss.str();\n        // Eliminar ceros finales y el punto decimal si no hay parte fraccionaria\n        // Esto puede ser delicado con std::scientific, as\u00ed que hay que ser cuidadosos.\n        // Para std::fixed:\n        if (s.find('.') != std::string::npos) {\n            s.erase(s.find_last_not_of('0') + 1, std::string::npos);\n            if (!s.empty() && s.back() == '.') s.pop_back();\n        }\n        return s.empty() ? \"0\" : s;\n    }\n}\n\n// --- evaluate_tree ---\ndouble evaluate_tree(const NodePtr& node, double x) {\n    if (!node) return std::nan(\"\");\n    switch (node->type) {\n        case NodeType::Constant: return node->value;\n        case NodeType::Variable: return x;\n        case NodeType::Operator: {\n            // Determine arity\n            bool is_unary = (node->op == 's' || node->op == 'c' || node->op == 'l' || node->op == 'e' || node->op == '!' || node->op == '_' || node->op == 'g');\n\n            double leftVal = evaluate_tree(node->left, x);\n            double rightVal = 0.0;\n            if (!is_unary) {\n                rightVal = evaluate_tree(node->right, x);\n            }\n\n            if (std::isnan(leftVal)) return std::nan(\"\");\n            if (!is_unary && std::isnan(rightVal)) return std::nan(\"\");\n\n            double result = std::nan(\"\");\n            try {\n                switch (node->op) {\n                    case '+': result = leftVal + rightVal; break;\n                    case '-': result = leftVal - rightVal; break;\n                    case '*': result = leftVal * rightVal; break;\n                    case '/':\n                        if (std::fabs(rightVal) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return INF;\n                        result = leftVal / rightVal;\n                        break;\n                    case '^':\n                        if (leftVal == 0.0 && rightVal == 0.0) result = 1.0;\n                        else if (leftVal == 0.0 && rightVal < 0.0) return INF;\n                        else if (leftVal < 0.0 && std::fabs(rightVal - std::round(rightVal)) > SIMPLIFY_NEAR_ZERO_TOLERANCE) return INF;\n                        else result = std::pow(leftVal, rightVal);\n                        break;\n                    case '%':\n                        if (std::fabs(rightVal) < SIMPLIFY_NEAR_ZERO_TOLERANCE) return INF;\n                        result = std::fmod(leftVal, rightVal);\n                        break;\n                    case 's': result = std::sin(leftVal); break;\n                    case 'c': result = std::cos(leftVal); break;\n                    case 'l': \n                        // Protected Log: log(|x|)\n                        if (std::abs(leftVal) <= 1e-9) return INF; \n                        result = std::log(std::abs(leftVal)); \n                        break;\n                    case 'e': \n                        if (leftVal > 700.0) return INF; // Overflow check\n                        result = std::exp(leftVal); \n                        break;\n                    case '!': \n                        // Protected Factorial/Gamma: tgamma(|x|+1)\n                        // Allow up to reasonable limit. \n                        // |leftVal| to handle negatives.\n                        if (std::abs(leftVal) > 170.0) return INF; // Overflow check\n                        result = std::tgamma(std::abs(leftVal) + 1.0); \n                        break;\n                    case '_': result = std::floor(leftVal); break;\n                    case 'g':\n                        // Protected LGamma: lgamma(|x|+1)\n                        // Always defined for positive arguments.\n                        result = std::lgamma(std::abs(leftVal) + 1.0); \n                        break;\n                    default: return std::nan(\"\");\n                }\n            } catch (const std::exception& e) { return INF; }\n            if (std::isinf(result)) return INF;\n            if (std::isnan(result)) return std::nan(\"\");\n            return result;\n        }\n        default: return std::nan(\"\");\n    }\n}\n\n// --- tree_to_string ---\nstd::string tree_to_string(const NodePtr& node) {\n     if (!node) return \"NULL\";\n     switch (node->type) {\n        case NodeType::Constant: return format_constant(node->value);\n        case NodeType::Variable: return \"x\";\n        case NodeType::Operator: {\n            NodePtr left_node = node->left;\n            std::string left_str = tree_to_string(left_node);\n            \n            // Check arity\n            bool is_unary = (node->op == 's' || node->op == 'c' || node->op == 'l' || node->op == 'e' || node->op == '!' || node->op == '_' || node->op == 'g');\n\n            if (is_unary) {\n                switch(node->op) {\n                    case 's': return \"sin(\" + left_str + \")\";\n                    case 'c': return \"cos(\" + left_str + \")\";\n                    case 'l': return \"log(\" + left_str + \")\";\n                    case 'e': return \"exp(\" + left_str + \")\";\n                    case '!': return \"(\" + left_str + \")!\"; // Postfix for factorial\n                    case '_': return \"floor(\" + left_str + \")\";\n                    case 'g': return \"lgamma(\" + left_str + \")\";\n                    default: return \"op(\" + left_str + \")\";\n                }\n            }\n\n            NodePtr right_node = node->right;\n            std::string right_str = tree_to_string(right_node);\n            char current_op = node->op;\n            bool right_is_neg_const = (right_node && right_node->type == NodeType::Constant && right_node->value < 0.0);\n            if (right_is_neg_const) {\n                double abs_right_val = std::fabs(right_node->value);\n                std::string abs_right_str = format_constant(abs_right_val);\n                if (node->op == '+') { current_op = '-'; right_str = abs_right_str; }\n                else if (node->op == '-') { current_op = '+'; right_str = abs_right_str; }\n            }\n            // Simplificar impresi\u00f3n de (0-A) a (-A)\n            if (left_node && left_node->type == NodeType::Constant && left_node->value == 0.0 && current_op == '-') {\n                 return \"(-\" + right_str + \")\";\n            }\n            return \"(\" + left_str + current_op + right_str + \")\";\n        }\n        default: return \"?\";\n    }\n}\n\n// --- tree_size ---\nint tree_size(const NodePtr& node) {\n    if (!node) return 0;\n    if (node->type == NodeType::Constant || node->type == NodeType::Variable) return 1;\n    if (node->type == NodeType::Operator) {\n        return 1 + tree_size(node->left) + tree_size(node->right);\n    }\n    return 0;\n}\n\n// --- clone_tree ---\nNodePtr clone_tree(const NodePtr& node) {\n    if (!node) return nullptr;\n    auto new_node = std::make_shared<Node>();\n    new_node->type = node->type;\n    new_node->value = node->value;\n    new_node->op = node->op;\n    new_node->left = clone_tree(node->left);\n    new_node->right = clone_tree(node->right);\n    return new_node;\n}\n\n// --- collect_node_ptrs ---\nvoid collect_node_ptrs(NodePtr& node, std::vector<NodePtr*>& vec) {\n    if (!node) return;\n    vec.push_back(&node);\n    if (node->type == NodeType::Operator) {\n        collect_node_ptrs(node->left, vec);\n        collect_node_ptrs(node->right, vec);\n    }\n}\n\n// --- get_rng ---\n// === OPTIMIZACI\u00d3N: RNG thread-local para evitar contenci\u00f3n en OpenMP ===\nstd::mt19937& get_rng() {\n    thread_local std::mt19937 local_rng(\n        std::random_device{}() ^ \n        static_cast<unsigned>(std::hash<std::thread::id>{}(std::this_thread::get_id()))\n    );\n    return local_rng;\n}\n\n\n// ============================================================\n// --- Parser de F\u00f3rmulas desde String (v4 - Parser Corregido) ---\n// ============================================================\n\n// Helper para obtener precedencia de operadores\nint get_precedence(char op) {\n    switch (op) {\n        case '+': case '-': return 1;\n        case '*': case '/': case '%': return 2;\n        case '^': return 3;\n        default: return 0;\n    }\n}\n\n// Helper para aplicar un operador binario\nNodePtr apply_binary_operation(NodePtr right, NodePtr left, char op) {\n    if (!left || !right) {\n        throw std::runtime_error(\"Error al aplicar operaci\u00f3n binaria '\" + std::string(1, op) + \"': operandos insuficientes.\");\n    }\n    auto node = std::make_shared<Node>(NodeType::Operator);\n    node->op = op;\n    node->left = left;\n    node->right = right;\n    return node;\n}\n\n// Funci\u00f3n principal para parsear la f\u00f3rmula\nNodePtr parse_formula_string(const std::string& formula_raw) {\n    std::string formula = formula_raw;\n    formula.erase(std::remove_if(formula.begin(), formula.end(), ::isspace), formula.end());\n    if (formula.empty()) throw std::runtime_error(\"La f\u00f3rmula est\u00e1 vac\u00eda.\");\n\n    std::stack<NodePtr> operand_stack;\n    std::stack<char> operator_stack;\n\n    // Funci\u00f3n interna para procesar operadores seg\u00fan precedencia y asociatividad\n    auto process_operators_by_precedence = [&](int current_precedence, char current_op_char = 0) {\n        // La asociatividad derecha para '^' significa que se procesa si el operador en la pila\n        // tiene MAYOR precedencia, no MAYOR O IGUAL.\n        bool is_right_associative = (current_op_char == '^');\n\n        while (!operator_stack.empty() && operator_stack.top() != '(') {\n            char top_op = operator_stack.top();\n            int top_precedence = get_precedence(top_op);\n\n            if (is_right_associative ? (top_precedence > current_precedence) : (top_precedence >= current_precedence)) {\n                operator_stack.pop(); // Sacar operador de la pila\n                if (operand_stack.size() < 2) throw std::runtime_error(\"Operandos insuficientes para operador '\" + std::string(1, top_op) + \"'.\");\n                NodePtr right = operand_stack.top(); operand_stack.pop();\n                NodePtr left = operand_stack.top(); operand_stack.pop();\n                operand_stack.push(apply_binary_operation(right, left, top_op));\n            } else {\n                break; // Parar si la precedencia es menor o si es asociativo a la derecha y es igual\n            }\n        }\n    };\n\n    bool last_token_was_operand = false;\n\n    for (int i = 0; i < formula.length(); /* Incremento manual */ ) {\n        char token = formula[i];\n\n        // --- A. Parsear N\u00fameros ---\n        bool starts_number = isdigit(token) || (token == '.' && i + 1 < formula.length() && isdigit(formula[i+1]));\n        if (starts_number) {\n             if (last_token_was_operand) { // Implicit multiplication\n                 process_operators_by_precedence(get_precedence('*'));\n                 operator_stack.push('*');\n                 last_token_was_operand = false;\n             }\n            std::string num_str;\n            if (token == '.') num_str += '0';\n            num_str += token;\n            i++;\n            while (i < formula.length() && (isdigit(formula[i]) || (formula[i] == '.' && num_str.find('.') == std::string::npos))) {\n                num_str += formula[i];\n                i++;\n            }\n            try {\n                double value = std::stod(num_str);\n                auto node = std::make_shared<Node>(NodeType::Constant); node->value = value;\n                operand_stack.push(node);\n                last_token_was_operand = true;\n            } catch (const std::invalid_argument& e) {\n                throw std::runtime_error(\"N\u00famero inv\u00e1lido (formato): '\" + num_str + \"' - \" + e.what());\n            } catch (const std::out_of_range& e) {\n                throw std::runtime_error(\"N\u00famero inv\u00e1lido (rango): '\" + num_str + \"' - \" + e.what());\n            }\n            continue;\n        }\n\n        // --- B. Parsear Funciones Unarias (l, g, sin, cos, exp, log, lgamma, floor, gamma) ---\n        // Map of function names to their operator characters\n        std::unordered_map<std::string, char> func_map = {\n            {\"sin\", 's'}, {\"cos\", 'c'}, {\"log\", 'l'}, {\"exp\", 'e'},\n            {\"floor\", '_'}, {\"lgamma\", 'g'}, {\"gamma\", '!'}, \n            {\"l\", 'l'}, {\"g\", 'g'}, {\"e\", 'e'}, {\"s\", 's'}, {\"c\", 'c'}\n        };\n        \n        // Try to match function names (check longer names first)\n        bool matched_func = false;\n        for (const auto& [func_name, func_op] : func_map) {\n            if (i + func_name.length() <= formula.length() && \n                formula.substr(i, func_name.length()) == func_name &&\n                (i + func_name.length() >= formula.length() || formula[i + func_name.length()] == '(')) {\n                \n                // Check if this is actually a function call (followed by '(')\n                size_t after_name = i + func_name.length();\n                if (after_name < formula.length() && formula[after_name] == '(') {\n                    if (last_token_was_operand) { // Implicit multiplication\n                        process_operators_by_precedence(get_precedence('*'));\n                        operator_stack.push('*');\n                        last_token_was_operand = false;\n                    }\n                    \n                    // Find the matching closing parenthesis\n                    int paren_count = 1;\n                    size_t arg_start = after_name + 1;\n                    size_t j = arg_start;\n                    while (j < formula.length() && paren_count > 0) {\n                        if (formula[j] == '(') paren_count++;\n                        else if (formula[j] == ')') paren_count--;\n                        j++;\n                    }\n                    if (paren_count != 0) {\n                        throw std::runtime_error(\"Par\u00e9ntesis sin cerrar en funci\u00f3n '\" + func_name + \"'.\");\n                    }\n                    size_t arg_end = j - 1; // Position of closing ')'\n                    \n                    // Extract and recursively parse the argument\n                    std::string arg_str = formula.substr(arg_start, arg_end - arg_start);\n                    NodePtr arg_tree = parse_formula_string(arg_str);\n                    \n                    // Create unary operator node\n                    auto func_node = std::make_shared<Node>(NodeType::Operator);\n                    func_node->op = func_op;\n                    func_node->left = arg_tree;\n                    func_node->right = nullptr;\n                    \n                    operand_stack.push(func_node);\n                    last_token_was_operand = true;\n                    i = j; // Skip past the closing ')'\n                    matched_func = true;\n                    break;\n                }\n            }\n        }\n        if (matched_func) continue;\n\n        // --- C. Parsear Variable 'x' ---\n        if (token == 'x') {\n            if (last_token_was_operand) { // Implicit multiplication\n                 process_operators_by_precedence(get_precedence('*'));\n                 operator_stack.push('*');\n                 last_token_was_operand = false;\n            }\n            auto node = std::make_shared<Node>(NodeType::Variable);\n            operand_stack.push(node);\n            last_token_was_operand = true;\n            i++;\n            continue;\n        }\n\n        // --- D. Parsear Par\u00e9ntesis de Apertura '(' ---\n        if (token == '(') {\n            if (last_token_was_operand) { // Implicit multiplication\n                 process_operators_by_precedence(get_precedence('*'));\n                 operator_stack.push('*');\n                 last_token_was_operand = false;\n            }\n            operator_stack.push('(');\n            last_token_was_operand = false;\n            i++;\n            continue;\n        }\n\n        // --- E. Parsear Par\u00e9ntesis de Cierre ')' ---\n        if (token == ')') {\n             if (!last_token_was_operand) {\n                  if (!operator_stack.empty() && operator_stack.top() == '(') throw std::runtime_error(\"Par\u00e9ntesis vac\u00edos '()' encontrados.\");\n                  else throw std::runtime_error(\"Se esperaba un operando antes de ')'.\");\n             }\n            while (!operator_stack.empty() && operator_stack.top() != '(') {\n                process_operators_by_precedence(0);\n            }\n            if (operator_stack.empty()) throw std::runtime_error(\"Par\u00e9ntesis ')' sin correspondiente '('.\");\n            operator_stack.pop(); // Sacar '('\n            last_token_was_operand = true;\n            i++;\n            continue;\n        }\n\n        // --- F. Parsear Operadores (+ - * / ^ %) ---\n        if (std::string(\"+-*/^%\").find(token) != std::string::npos) {\n            // Manejar '-' unario vs binario\n            if (token == '-' && !last_token_was_operand) {\n                // Es un '-' unario. Insertar un 0 como operando izquierdo impl\u00edcito.\n                // Esto permite tratar el '-' como un operador binario normal.\n                auto zero_node = std::make_shared<Node>(NodeType::Constant); zero_node->value = 0.0;\n                operand_stack.push(zero_node);\n                // No cambiar last_token_was_operand a true, ya que el 0 impl\u00edcito\n                // es solo para el operador unario y no un operando \"real\" previo.\n                // Si hubiera una multiplicaci\u00f3n impl\u00edcita (ej. \"2-x\"), ya se habr\u00eda manejado.\n            }\n            // Ignorar '+' unario (no afecta el valor, no necesita un 0 impl\u00edcito)\n            else if (token == '+' && !last_token_was_operand) {\n                // No hacer nada, simplemente avanzar al siguiente token\n                i++;\n                continue;\n            }\n            \n            // Operador binario normal\n            if (!last_token_was_operand && (token == '*' || token == '/' || token == '^' || token == '%')) {\n                throw std::runtime_error(\"Operador binario '\" + std::string(1, token) + \"' inesperado. Se esperaba operando.\");\n            }\n\n            // Procesar operadores en la pila con mayor o igual precedencia (o solo mayor para asociativos a derecha)\n            process_operators_by_precedence(get_precedence(token), token);\n            operator_stack.push(token);\n            last_token_was_operand = false; // Despu\u00e9s de un operador, se espera un operando\n            i++;\n            continue;\n        }\n\n        // --- G. Token Desconocido ---\n        throw std::runtime_error(\"Token desconocido en la f\u00f3rmula: '\" + std::string(1, token) + \"'\");\n\n    } // Fin del bucle for\n\n    // --- H. Procesamiento Final despu\u00e9s del bucle ---\n    while (!operator_stack.empty()) {\n        if (operator_stack.top() == '(') throw std::runtime_error(\"Par\u00e9ntesis '(' sin cerrar al final.\");\n        // Procesar todos los operadores restantes en la pila\n        process_operators_by_precedence(0); // 0 como precedencia m\u00ednima para forzar el procesamiento\n    }\n\n    // Verificaci\u00f3n final de la pila de operandos\n    if (operand_stack.size() != 1) {\n         if (operand_stack.empty() && formula.length() > 0) throw std::runtime_error(\"Error: No se gener\u00f3 ning\u00fan resultado del parseo. F\u00f3rmula inv\u00e1lida?\");\n         else if (operand_stack.size() > 1) throw std::runtime_error(\"Error en la estructura final (operandos restantes: \" + std::to_string(operand_stack.size()) + \"). Verifique operadores.\");\n         else throw std::runtime_error(\"Error desconocido al finalizar el parseo.\");\n    }\n\n    return operand_stack.top();\n}\n",
  "src/ExpressionTree.h": "#ifndef EXPRESSIONTREE_H\n#define EXPRESSIONTREE_H\n\n#include <memory>\n#include <string>\n#include <vector>\n#include <stdexcept> // Para std::runtime_error\n\n// Forward declaration\nstruct Node;\n\n// Use shared_ptr for automatic memory management\nusing NodePtr = std::shared_ptr<Node>;\n\nenum class NodeType { Constant, Variable, Operator };\n\nstruct Node {\n    NodeType type;\n    double value = 0.0;             // If type == Constant\n    char op = 0;                    // If type == Operator: '+', '-', '*', '/', '^'\n    NodePtr left = nullptr;         // Children (for Operators)\n    NodePtr right = nullptr;\n\n    // Constructor for convenience\n    Node(NodeType t = NodeType::Constant) : type(t) {}\n};\n\n// Core Tree Functions\ndouble evaluate_tree(const NodePtr& node, double x);\nstd::string tree_to_string(const NodePtr& node);\nint tree_size(const NodePtr& node);\nNodePtr clone_tree(const NodePtr& node);\n\n// Helper for mutation/crossover\nvoid collect_node_ptrs(NodePtr& node, std::vector<NodePtr*>& vec);\n\n// --- NUEVO: Funci\u00f3n para parsear una f\u00f3rmula desde string ---\n// Parsea una f\u00f3rmula en notaci\u00f3n infija simple (con par\u00e9ntesis).\n// Lanza std::runtime_error si hay error de sintaxis.\nNodePtr parse_formula_string(const std::string& formula);\n\n\n#endif // EXPRESSIONTREE_H\n",
  "src/Fitness.cpp": "#include \"Fitness.h\"\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n#include \"FitnessGPU.cuh\" // Include for GPU fitness evaluation\n#endif\n#include \"Globals.h\" // Necesario para constantes globales e INF\n#include \"ExpressionTree.h\" // Necesario para tree_to_string\n#include <cmath>\n#include <limits>\n#include <vector>\n#include <numeric>\n#include <iostream> // Para std::cerr en caso de error futuro\n#include <iomanip>  // Para std::fixed/scientific si se necesita en errores\n\n// Calculates the raw fitness using global parameters.\n// This function will now dispatch to GPU if USE_GPU_ACCELERATION is enabled.\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\ndouble calculate_raw_fitness(const NodePtr& tree,\n                             const std::vector<double>& targets,\n                             const std::vector<double>& x_values,\n                             double* d_targets, double* d_x_values) {\n    return evaluate_fitness_gpu(tree, targets, x_values, d_targets, d_x_values);\n}\n#else\ndouble calculate_raw_fitness(const NodePtr& tree,\n                             const std::vector<double>& targets,\n                             const std::vector<double>& x_values) {\n    if (x_values.size() != targets.size() || x_values.empty()) return INF;\n\n    double error_sum_pow13 = 0.0; // Solo si USE_RMSE_FITNESS = false\n    double sum_sq_error = 0.0;\n    double total_weight = 0.0; // Para normalizar el fitness ponderado\n    bool all_precise = true;\n    size_t num_points = x_values.size();\n    bool calculation_failed = false; // Flag para detectar INF/NaN\n\n    for (size_t i = 0; i < num_points; ++i) {\n        double predicted_val = evaluate_tree(tree, x_values[i]);\n\n        // Comprobar si la evaluaci\u00f3n fall\u00f3 (INF o NaN)\n        if (std::isnan(predicted_val) || std::isinf(predicted_val)) {\n            calculation_failed = true;\n            break; // Salir del bucle si la evaluaci\u00f3n falla para un punto\n        }\n\n        double target_val = targets[i];\n        double diff = predicted_val - target_val;\n        double abs_diff = std::fabs(diff);\n\n        if (abs_diff >= FITNESS_PRECISION_THRESHOLD) all_precise = false;\n\n        // --- PESO PARA FITNESS PONDERADO ---\n        // Hace que los \u00faltimos puntos (N altos) valgan much\u00edsimo m\u00e1s.\n        // Esto destruye a los polinomios porque fallan al final.\n        double weight = 1.0;\n        if (USE_WEIGHTED_FITNESS) {\n            // Peso exponencial: m\u00e1s agresivo para penalizar errores en N altos\n            weight = std::exp(static_cast<double>(i) * WEIGHTED_FITNESS_EXPONENT);\n        }\n        total_weight += weight;\n\n        // Acumular error para ambas m\u00e9tricas (si aplica)\n        if (!USE_RMSE_FITNESS) {\n             error_sum_pow13 += std::pow(abs_diff, FITNESS_ORIGINAL_POWER) * weight;\n        }\n\n        // Calcular y acumular error cuadr\u00e1tico PONDERADO\n        double sq_diff = diff * diff;\n        sum_sq_error += sq_diff * weight;\n\n        // Control de desbordamiento/Infinito en la suma\n        if (std::isinf(sum_sq_error) || (error_sum_pow13 >= INF / 10.0 && !USE_RMSE_FITNESS)) {\n            calculation_failed = true;\n            break;\n        }\n    } // Fin bucle for puntos\n\n    // Si la evaluaci\u00f3n o suma fall\u00f3 en alg\u00fan punto, devolver INF\n    if (calculation_failed) {\n        return INF;\n    }\n\n    // Seleccionar m\u00e9trica de error crudo\n    double raw_error;\n    if (USE_RMSE_FITNESS) {\n        if (num_points == 0 || total_weight == 0.0) return INF;\n        // MSE ponderado: normalizar por suma de pesos, no por num_points\n        double mse = sum_sq_error / total_weight;\n        if (std::isinf(mse) || std::isnan(mse) || mse < 0) {\n             raw_error = INF;\n        } else {\n             raw_error = std::sqrt(mse); // Calcular RMSE ponderado\n        }\n    } else {\n        raw_error = error_sum_pow13;\n    }\n\n    // Comprobar si el error crudo es inv\u00e1lido\n    if (std::isnan(raw_error) || std::isinf(raw_error) || raw_error < 0) {\n         return INF;\n    }\n\n    // Aplicar bonus de precisi\u00f3n si todos los puntos estaban dentro del umbral\n    if (all_precise) {\n         raw_error *= FITNESS_PRECISION_BONUS;\n    }\n\n    return raw_error; // Devolver el error crudo (sin penalizaci\u00f3n por complejidad a\u00fan)\n}\n#endif // USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n\n// Calcula el fitness final usando par\u00e1metros globales.\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\ndouble evaluate_fitness(const NodePtr& tree,\n                        const std::vector<double>& targets,\n                        const std::vector<double>& x_values,\n                        double* d_targets, double* d_x_values) {\n    double raw_fitness = calculate_raw_fitness(tree, targets, x_values, d_targets, d_x_values);\n#else\ndouble evaluate_fitness(const NodePtr& tree,\n                        const std::vector<double>& targets,\n                        const std::vector<double>& x_values) {\n    double raw_fitness = calculate_raw_fitness(tree, targets, x_values);\n#endif\n\n    if (raw_fitness >= INF / 10.0) {\n         return INF; // Si el error crudo es infinito, el fitness final es infinito\n    }\n\n    // Penalizaci\u00f3n por complejidad\n    double complexity = static_cast<double>(tree_size(tree));\n    double penalty = complexity * COMPLEXITY_PENALTY_FACTOR; // Usa constante global\n\n    // Aplicar penalizaci\u00f3n multiplicativa\n    double final_fitness = raw_fitness * (1.0 + penalty);\n\n    // Comprobaciones finales\n    if (std::isnan(final_fitness) || std::isinf(final_fitness) || final_fitness < 0) {\n         return INF;\n    }\n\n    return final_fitness;\n}\n",
  "src/Fitness.h": "#ifndef FITNESS_H\n#define FITNESS_H\n\n#include \"ExpressionTree.h\"\n#include <vector>\n\n// Calculates raw fitness based on target matching\n// Lower is better. Returns INF if evaluation results in NaN/Inf.\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\ndouble calculate_raw_fitness(const NodePtr& tree,\n                             const std::vector<double>& targets,\n                             const std::vector<double>& x_values,\n                             double* d_targets, double* d_x_values);\n#else\ndouble calculate_raw_fitness(const NodePtr& tree,\n                             const std::vector<double>& targets,\n                             const std::vector<double>& x_values);\n#endif\n\n// Calculates final fitness including complexity penalty\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\ndouble evaluate_fitness(const NodePtr& tree,\n                        const std::vector<double>& targets,\n                        const std::vector<double>& x_values,\n                        double* d_targets, double* d_x_values);\n#else\ndouble evaluate_fitness(const NodePtr& tree,\n                        const std::vector<double>& targets,\n                        const std::vector<double>& x_values);\n#endif\n\n#endif // FITNESS_H\n",
  "src/FitnessGPU.cu": "#include \"FitnessGPU.cuh\"\n#include \"Globals.h\"\n#include <cuda_runtime.h>\n#include <math.h>\n#include <vector>\n#include <iostream>\n#include <cstdio>\n\n// Error checking macro\n#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }\ninline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)\n{\n   if (code != cudaSuccess) \n   {\n      fprintf(stderr, \"GPUassert: %s %s %d\\n\", cudaGetErrorString(code), file, line);\n      if (abort) exit(code);\n   }\n}\n\n// Helper function to linearize the tree into a post-order array\nvoid linearize_tree(const NodePtr& node, std::vector<LinearGpuNode>& linear_tree) {\n    if (!node) {\n        return;\n    }\n    linearize_tree(node->left, linear_tree);\n    linearize_tree(node->right, linear_tree);\n    linear_tree.push_back({node->type, node->value, node->op});\n}\n\n#if USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n\n// Constant for large finite value\n#define GPU_MAX_DOUBLE 1e308\n\n// Single Tree Evaluation Kernel (Legacy/Single Use)\n__global__ void calculate_raw_fitness_kernel(const LinearGpuNode* d_linear_tree,\n                                             int tree_size,\n                                             const double* d_targets,\n                                             const double* d_x_values,\n                                             size_t num_points,\n                                             double* d_raw_fitness_results) {\n    // Shared memory optimization: Load tree into shared memory\n    extern __shared__ LinearGpuNode s_linear_tree[];\n\n    // Cooperative load\n    for (int i = threadIdx.x; i < tree_size; i += blockDim.x) {\n        s_linear_tree[i] = d_linear_tree[i];\n    }\n    __syncthreads();\n\n    int idx = blockIdx.x * blockDim.x + threadIdx.x;\n\n    if (idx < num_points) {\n        double x_val = d_x_values[idx];\n        double stack[64]; // Max tree depth\n        int stack_top = -1;\n\n        for (int i = 0; i < tree_size; ++i) {\n            LinearGpuNode node = s_linear_tree[i]; // Access from shared memory\n            if (node.type == NodeType::Constant) {\n                stack[++stack_top] = node.value;\n            } else if (node.type == NodeType::Variable) {\n                stack[++stack_top] = x_val;\n            } else if (node.type == NodeType::Operator) {\n                double right = stack[stack_top--];\n                double left = stack[stack_top--];\n                double result;\n                switch (node.op) {\n                    case '+': result = left + right; break;\n                    case '-': result = left - right; break;\n                    case '*': result = left * right; break;\n                    case '/':\n                        if (fabs(right) < 1e-9) { // Avoid division by zero\n                            result = GPU_MAX_DOUBLE; \n                        } else {\n                            result = left / right;\n                        }\n                        break;\n                    default: result = NAN; break;\n                }\n                stack[++stack_top] = result;\n            }\n        }\n\n        double predicted_val = (stack_top == 0) ? stack[0] : NAN;\n\n        if (isnan(predicted_val) || isinf(predicted_val)) {\n            d_raw_fitness_results[idx] = GPU_MAX_DOUBLE; \n        } else {\n            double diff = predicted_val - d_targets[idx];\n            d_raw_fitness_results[idx] = diff * diff;\n        }\n    }\n}\n\n// CUDA kernel for parallel reduction (summation)\n__global__ void reduce_sum_kernel(double* d_data, int N) {\n    extern __shared__ double sdata[]; // Shared memory for reduction\n\n    unsigned int tid = threadIdx.x;\n    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;\n\n    sdata[tid] = (i < N) ? d_data[i] : 0.0; // Load data into shared memory\n\n    __syncthreads(); // Synchronize threads in block\n\n    // Perform reduction in shared memory\n    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {\n        if (tid < s) {\n            sdata[tid] += sdata[tid + s];\n        }\n        __syncthreads();\n    }\n\n    if (tid == 0) { // Write result back to global memory (first element of block)\n        d_data[blockIdx.x] = sdata[0];\n    }\n}\n\n\n// --- New Batch Kernel ---\n// Evaluates one tree per thread across all data points\n__global__ void evaluate_population_kernel(const LinearGpuNode* d_all_nodes,\n                                           const int* d_offsets,\n                                           const int* d_sizes,\n                                           int pop_size,\n                                           const double* d_targets,\n                                           const double* d_x_values,\n                                           int num_points,\n                                           double* d_results) {\n    int idx = blockIdx.x * blockDim.x + threadIdx.x;\n\n    if (idx < pop_size) {\n        int offset = d_offsets[idx];\n        int size = d_sizes[idx];\n        double sum_sq_error = 0.0;\n        bool valid = true;\n\n        for (int p = 0; p < num_points; ++p) {\n            double x_val = d_x_values[p];\n            double stack[64]; \n            int stack_top = -1;\n\n            // Simple interpreter\n            for (int i = 0; i < size; ++i) {\n                LinearGpuNode node = d_all_nodes[offset + i];\n                if (node.type == NodeType::Constant) {\n                    stack[++stack_top] = node.value;\n                } else if (node.type == NodeType::Variable) {\n                    stack[++stack_top] = x_val;\n                } else if (node.type == NodeType::Operator) {\n                    // Safety check index\n                    if (stack_top < 1) { valid = false; break; }\n\n                    double right = stack[stack_top--];\n                    double left = stack[stack_top--];\n                    double result;\n                    switch (node.op) {\n                        case '+': result = left + right; break;\n                        case '-': result = left - right; break;\n                        case '*': result = left * right; break;\n                        case '/':\n                            if (fabs(right) < 1e-9) { \n                                result = GPU_MAX_DOUBLE; \n                            } else {\n                                result = left / right;\n                            }\n                            break;\n                        default: result = NAN; break;\n                    }\n                    stack[++stack_top] = result;\n                }\n            }\n\n            if (!valid || stack_top != 0) {\n                sum_sq_error = GPU_MAX_DOUBLE;\n                break;\n            }\n\n            double predicted_val = stack[0];\n            if (isnan(predicted_val) || isinf(predicted_val)) {\n                sum_sq_error = GPU_MAX_DOUBLE;\n                break;\n            }\n\n            double diff = predicted_val - d_targets[p];\n            sum_sq_error += diff * diff;\n        }\n\n        d_results[idx] = sum_sq_error;\n    }\n}\n\n\n// Host-side wrapper function to launch the CUDA kernel\ndouble evaluate_fitness_gpu(NodePtr tree,\n                            const std::vector<double>& targets,\n                            const std::vector<double>& x_values,\n                            double* d_targets, double* d_x_values) {\n    if (x_values.size() != targets.size() || x_values.empty()) return INF;\n\n    // Linearize the tree\n    std::vector<LinearGpuNode> h_linear_tree;\n    linearize_tree(tree, h_linear_tree);\n    int tree_size = h_linear_tree.size();\n\n    if (tree_size == 0) {\n        return INF;\n    }\n\n    size_t num_points = x_values.size();\n    LinearGpuNode* d_linear_tree;\n    double* d_raw_fitness_results; // This will hold individual errors and then the final sum\n\n    gpuErrchk(cudaMalloc((void**)&d_linear_tree, tree_size * sizeof(LinearGpuNode)));\n    gpuErrchk(cudaMalloc((void**)&d_raw_fitness_results, num_points * sizeof(double)));\n\n    gpuErrchk(cudaMemcpy(d_linear_tree, h_linear_tree.data(), tree_size * sizeof(LinearGpuNode), cudaMemcpyHostToDevice));\n\n    int threadsPerBlock = 256;\n    int blocksPerGrid = (num_points + threadsPerBlock - 1) / threadsPerBlock;\n\n    // Launch kernel to calculate individual squared errors\n    size_t shared_mem_size = tree_size * sizeof(LinearGpuNode);\n    calculate_raw_fitness_kernel<<<blocksPerGrid, threadsPerBlock, shared_mem_size>>>(\n        d_linear_tree, tree_size, d_targets, d_x_values, num_points, d_raw_fitness_results\n    );\n    gpuErrchk(cudaPeekAtLastError());\n    gpuErrchk(cudaDeviceSynchronize()); // Ensure kernel completes before reduction\n\n    // --- Perform reduction on the GPU ---\n    int current_size = num_points;\n    while (current_size > 1) {\n        int next_blocks_per_grid = (current_size + threadsPerBlock - 1) / threadsPerBlock;\n        // Use shared memory for reduction, size is threadsPerBlock * sizeof(double)\n        reduce_sum_kernel<<<next_blocks_per_grid, threadsPerBlock, threadsPerBlock * sizeof(double)>>>(\n            d_raw_fitness_results, current_size\n        );\n        gpuErrchk(cudaPeekAtLastError());\n        gpuErrchk(cudaDeviceSynchronize()); // Ensure reduction step completes\n        current_size = next_blocks_per_grid; // The result is in the first `next_blocks_per_grid` elements\n    }\n\n    double sum_sq_error_gpu = 0.0;\n    gpuErrchk(cudaMemcpy(&sum_sq_error_gpu, d_raw_fitness_results, sizeof(double), cudaMemcpyDeviceToHost));\n\n    gpuErrchk(cudaFree(d_linear_tree));\n    gpuErrchk(cudaFree(d_raw_fitness_results));\n\n    // Check for invalid results (propagated from kernel)\n    if (isinf(sum_sq_error_gpu) || isnan(sum_sq_error_gpu)) {\n        return INF;\n    }\n\n    double raw_fitness;\n    if (USE_RMSE_FITNESS) {\n        if (num_points == 0) return INF;\n        double mse = sum_sq_error_gpu / num_points;\n        raw_fitness = sqrt(mse);\n    } else {\n        raw_fitness = sum_sq_error_gpu;\n    }\n\n    double complexity = static_cast<double>(::tree_size(tree));\n    double penalty = complexity * COMPLEXITY_PENALTY_FACTOR;\n    double final_fitness = raw_fitness * (1.0 + penalty);\n\n    if (isnan(final_fitness) || isinf(final_fitness) || final_fitness < 0) {\n        return INF;\n    }\n\n    return final_fitness;\n}\n\nvoid evaluate_population_gpu(const std::vector<LinearGpuNode>& all_nodes,\n                             const std::vector<int>& tree_offsets,\n                             const std::vector<int>& tree_sizes,\n                             const std::vector<double>& targets,\n                             const std::vector<double>& x_values,\n                             std::vector<double>& results,\n                             double* d_targets, double* d_x_values,\n                             void*& d_nodes_ptr, size_t& d_nodes_cap,\n                             void*& d_offsets_ptr, void*& d_sizes_ptr, void*& d_results_ptr, size_t& d_pop_cap) {\n    \n    int pop_size = tree_offsets.size();\n    if (pop_size == 0) return;\n\n    size_t total_nodes = all_nodes.size();\n    int num_points = x_values.size();\n\n    // Buffer Management for Nodes\n    if (total_nodes > d_nodes_cap) {\n        if (d_nodes_ptr) gpuErrchk(cudaFree(d_nodes_ptr));\n        size_t new_cap = total_nodes * 1.5; // Growth factor\n        gpuErrchk(cudaMalloc(&d_nodes_ptr, new_cap * sizeof(LinearGpuNode)));\n        d_nodes_cap = new_cap;\n    }\n\n    // Buffer Management for Population Arrays\n    if (pop_size > d_pop_cap) {\n        if (d_offsets_ptr) gpuErrchk(cudaFree(d_offsets_ptr));\n        if (d_sizes_ptr) gpuErrchk(cudaFree(d_sizes_ptr));\n        if (d_results_ptr) gpuErrchk(cudaFree(d_results_ptr));\n        \n        size_t new_cap = pop_size * 1.5;\n        gpuErrchk(cudaMalloc(&d_offsets_ptr, new_cap * sizeof(int)));\n        gpuErrchk(cudaMalloc(&d_sizes_ptr, new_cap * sizeof(int)));\n        gpuErrchk(cudaMalloc(&d_results_ptr, new_cap * sizeof(double)));\n        d_pop_cap = new_cap;\n    }\n\n    LinearGpuNode* d_all_nodes = (LinearGpuNode*)d_nodes_ptr;\n    int* d_offsets = (int*)d_offsets_ptr;\n    int* d_sizes = (int*)d_sizes_ptr;\n    double* d_results = (double*)d_results_ptr;\n\n    gpuErrchk(cudaMemcpy(d_all_nodes, all_nodes.data(), total_nodes * sizeof(LinearGpuNode), cudaMemcpyHostToDevice));\n    gpuErrchk(cudaMemcpy(d_offsets, tree_offsets.data(), pop_size * sizeof(int), cudaMemcpyHostToDevice));\n    gpuErrchk(cudaMemcpy(d_sizes, tree_sizes.data(), pop_size * sizeof(int), cudaMemcpyHostToDevice));\n\n    int threadsPerBlock = 256;\n    int blocksPerGrid = (pop_size + threadsPerBlock - 1) / threadsPerBlock;\n\n    evaluate_population_kernel<<<blocksPerGrid, threadsPerBlock>>>(\n        d_all_nodes, d_offsets, d_sizes, pop_size, d_targets, d_x_values, num_points, d_results\n    );\n    gpuErrchk(cudaPeekAtLastError());\n\n    // Synchronize and copy back\n    gpuErrchk(cudaDeviceSynchronize());\n    \n    gpuErrchk(cudaMemcpy(results.data(), d_results, pop_size * sizeof(double), cudaMemcpyDeviceToHost));\n}\n\n#endif // USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n",
  "src/FitnessGPU.cuh": "#ifndef FITNESS_GPU_CUH\n#define FITNESS_GPU_CUH\n\n#include <vector>\n#include <memory> // For NodePtr in the host-side wrapper\n#include \"ExpressionTree.h\" // For NodeType enum and original Node structure (host-side)\n#include \"Globals.h\" // For INF, USE_RMSE_FITNESS, COMPLEXITY_PENALTY_FACTOR etc.\n\n// Forward declaration for host-side NodePtr\nstruct Node;\nusing NodePtr = std::shared_ptr<Node>;\n\n// A simplified node structure for the linearized tree on the GPU\nstruct LinearGpuNode {\n    NodeType type;\n    double value;\n    char op;\n};\n\n// Helper function to linearize the tree into a post-order array\nvoid linearize_tree(const NodePtr& node, std::vector<LinearGpuNode>& linear_tree);\n\n// Host-side wrapper for launching CUDA kernel\n#if USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\ndouble evaluate_fitness_gpu(NodePtr tree,\n                            const std::vector<double>& targets,\n                            const std::vector<double>& x_values,\n                            double* d_targets, double* d_x_values);\n\n// Batch evaluation function with persistent buffers\nvoid evaluate_population_gpu(const std::vector<LinearGpuNode>& all_nodes,\n                             const std::vector<int>& tree_offsets,\n                             const std::vector<int>& tree_sizes,\n                             const std::vector<double>& targets,\n                             const std::vector<double>& x_values,\n                             std::vector<double>& results,\n                             double* d_targets, double* d_x_values,\n                             void*& d_nodes_ptr, size_t& d_nodes_cap,\n                             void*& d_offsets_ptr, void*& d_sizes_ptr, void*& d_results_ptr, size_t& d_pop_cap);\n#endif\n\n#endif // FITNESS_GPU_CUH\n",
  "src/GeneticAlgorithm.cpp": "#include \"GeneticAlgorithm.h\"\n#include \"Globals.h\"\n#include \"Fitness.h\"\n#include \"AdvancedFeatures.h\" // Incluir este para DomainConstraints::\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n#include \"FitnessGPU.cuh\"     // Para funciones de GPU\n#include <cuda_runtime.h>     // Para CUDA runtime\n#endif\n#include <iostream>\n#include <algorithm>\n#include <vector>\n#include <cmath>\n#include <omp.h>\n#include <iomanip>\n#include <iterator>\n#include <chrono>\n\n// --- Constructor (Modificado para que evaluate_population procese todo) ---\nGeneticAlgorithm::GeneticAlgorithm(const std::vector<double>& targets_ref,\n                                     const std::vector<double>& x_values_ref,\n                                     int total_pop,\n                                     int gens,\n                                     int n_islands)\n    : targets(targets_ref),\n      x_values(x_values_ref),\n      total_population_size(total_pop),\n      generations(gens),\n      num_islands(n_islands),\n      overall_best_fitness(INF),\n      last_overall_best_fitness(INF),\n      generation_last_improvement(0)\n{\n    // Validar y ajustar n\u00famero de islas y poblaci\u00f3n por isla\n    if (this->num_islands <= 0) this->num_islands = 1;\n    pop_per_island = this->total_population_size / this->num_islands;\n    if (pop_per_island < MIN_POP_PER_ISLAND) {\n        pop_per_island = MIN_POP_PER_ISLAND;\n        this->num_islands = this->total_population_size / pop_per_island;\n        if (this->num_islands == 0) this->num_islands = 1;\n        std::cerr << \"Warning: Adjusted number of islands to \" << this->num_islands\n                  << \" for minimum population size per island (\" << pop_per_island <<\").\" << std::endl;\n    }\n    this->total_population_size = this->num_islands * pop_per_island;\n    std::cout << \"Info: Running with \" << this->num_islands << \" islands, \"\n              << pop_per_island << \" individuals per island.\" << std::endl;\n\n    // Crear las islas\n    islands.reserve(this->num_islands);\n    for (int i = 0; i < this->num_islands; ++i) {\n        try {\n            islands.push_back(std::make_unique<Island>(i, pop_per_island));\n        }\n        catch (const std::exception& e) { std::cerr << \"[ERROR] Creating Island \" << i << \": \" << e.what() << std::endl; throw; }\n        catch (...) { std::cerr << \"[ERROR] Unknown exception creating island \" << i << std::endl; throw; }\n    }\n\n    // --- ELIMINADO: Bloque de evaluaci\u00f3n especial para f\u00f3rmula inyectada ---\n    // if (USE_INITIAL_FORMULA) { ... }\n\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n    // Asignar memoria en la GPU y copiar datos\n    size_t targets_size = targets.size() * sizeof(double);\n    size_t x_values_size = x_values.size() * sizeof(double);\n\n    cudaError_t err_t = cudaMalloc(&d_targets, targets_size);\n    cudaError_t err_x = cudaMalloc(&d_x_values, x_values_size);\n\n    if (err_t != cudaSuccess || err_x != cudaSuccess) {\n        std::cerr << \"[ERROR] CUDA memory allocation failed: \"\n                  << cudaGetErrorString(err_t) << \" | \" << cudaGetErrorString(err_x) << std::endl;\n        // Considerar lanzar una excepci\u00f3n o manejar el error adecuadamente\n        throw std::runtime_error(\"CUDA memory allocation failed\");\n    }\n\n    cudaMemcpy(d_targets, targets.data(), targets_size, cudaMemcpyHostToDevice);\n    cudaMemcpy(d_x_values, x_values.data(), x_values_size, cudaMemcpyHostToDevice);\n#endif\n\n     // Evaluaci\u00f3n inicial de TODA la poblaci\u00f3n (incluyendo la inyectada)\n     // La funci\u00f3n evaluate_population ahora simplificar\u00e1 y evaluar\u00e1 a todos.\n     std::cout << \"Evaluating initial population (simplifying all)...\" << std::endl;\n     #pragma omp parallel for\n     for (int i = 0; i < islands.size(); ++i) {\n        evaluate_population(*islands[i]);\n     }\n\n     // Actualizar el mejor global inicial (en serie)\n     overall_best_fitness = INF;\n     overall_best_tree = nullptr;\n     int initial_best_island = -1;\n     int initial_best_idx = -1;\n\n     for (int i = 0; i < islands.size(); ++i) {\n        for(int j=0; j < islands[i]->population.size(); ++j) {\n            const auto& ind = islands[i]->population[j];\n            if (ind.tree && ind.fitness_valid && ind.fitness < overall_best_fitness) {\n                overall_best_fitness = ind.fitness;\n                initial_best_island = i;\n                initial_best_idx = j;\n            }\n        }\n     }\n     if(initial_best_island != -1 && initial_best_idx != -1) {\n         overall_best_tree = clone_tree(islands[initial_best_island]->population[initial_best_idx].tree);\n     }\n\n     last_overall_best_fitness = overall_best_fitness;\n     generation_last_improvement = 0;\n     std::cout << \"Initial best fitness: \" << std::scientific << overall_best_fitness << std::fixed << std::endl;\n     if (overall_best_tree) {\n          std::cout << \"Initial best formula size: \" << tree_size(overall_best_tree) << std::endl;\n          std::cout << \"Initial best formula: \" << tree_to_string(overall_best_tree) << std::endl;\n          // Nota para saber si el mejor inicial fue la f\u00f3rmula inyectada (ahora simplificada)\n          if (USE_INITIAL_FORMULA && initial_best_island != -1 && initial_best_idx == 0) {\n               std::cout << \"   (Note: Initial best is the (simplified) injected formula from Island \" << initial_best_island << \")\" << std::endl;\n          } else if (initial_best_island != -1) {\n               std::cout << \"   (Note: Initial best found in Island \" << initial_best_island << \", Index \" << initial_best_idx << \")\" << std::endl;\n          }\n      } else { std::cout << \"No valid initial solution found (all fitness INF?).\" << std::endl; }\n     std::cout << \"----------------------------------------\" << std::endl;\n}\n\nGeneticAlgorithm::~GeneticAlgorithm() {\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n    if (d_targets) {\n        cudaFree(d_targets);\n        d_targets = nullptr;\n    }\n    if (d_x_values) {\n        cudaFree(d_x_values);\n        d_x_values = nullptr;\n    }\n#endif\n    // El destructor de std::unique_ptr en 'islands' se encarga de liberar la memoria de las islas.\n    // 'overall_best_tree' es un NodePtr. Si es un smart pointer (como std::unique_ptr<Node>),\n    // su memoria se liberar\u00e1 autom\u00e1ticamente. Si es un puntero crudo, necesitar\u00eda una funci\u00f3n delete_tree.\n    // Asumiendo que NodePtr es un smart pointer o que la liberaci\u00f3n se maneja en otro lugar,\n    // o que un \u00e1rbol nulo al final no causa fugas si no fue asignado con 'new'.\n    // Si NodePtr es un puntero crudo y se asigna con 'new' en clone_tree, entonces\n    // delete_tree(overall_best_tree) ser\u00eda necesario aqu\u00ed.\n    // Por ahora, se deja vac\u00edo, asumiendo manejo autom\u00e1tico o externo.\n}\n\nvoid GeneticAlgorithm::evaluate_population(Island& island) {\n    int pop_size = island.population.size();\n    if (pop_size == 0) return;\n\n    // 1. Simplify trees (CPU Parallel)\n    // We do this first so we only send simplified trees to GPU\n    #pragma omp parallel for schedule(dynamic)\n    for (int i = 0; i < pop_size; ++i) {\n        Individual& ind = island.population[i];\n        if (ind.tree) {\n            ind.tree = DomainConstraints::fix_or_simplify(ind.tree);\n        }\n    }\n\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n    // 2. Prepare for Batch GPU Evaluation\n    std::vector<LinearGpuNode> all_nodes;\n    std::vector<int> tree_offsets;\n    std::vector<int> tree_sizes;\n    \n    // Reserve memory to avoid reallocations (Optimization)\n    // Assuming average tree size is around 20-30 nodes. \n    // This dramatically reduces CPU overhead during linearization.\n    all_nodes.reserve(pop_size * 30); \n    tree_offsets.reserve(pop_size);\n    tree_sizes.reserve(pop_size);\n    \n    // We need map back to original index because some trees might be null\n    std::vector<int> valid_indices; \n    valid_indices.reserve(pop_size);\n\n    for (int i = 0; i < pop_size; ++i) {\n        if (island.population[i].tree) {\n            int start_offset = all_nodes.size();\n            linearize_tree(island.population[i].tree, all_nodes);\n            int size = all_nodes.size() - start_offset;\n            \n            if (size > 0) {\n                tree_offsets.push_back(start_offset);\n                tree_sizes.push_back(size);\n                valid_indices.push_back(i);\n            } else {\n                 island.population[i].fitness = INF;\n                 island.population[i].fitness_valid = true;\n            }\n        } else {\n             island.population[i].fitness = INF;\n             island.population[i].fitness_valid = true;\n        }\n    }\n\n    if (valid_indices.empty()) return;\n\n    // 3. call GPU Batch (d_targets and d_x_values already exist)\n    std::vector<double> raw_results(valid_indices.size());\n    evaluate_population_gpu(all_nodes, tree_offsets, tree_sizes, targets, x_values, raw_results, d_targets, d_x_values,\n                            island.d_nodes, island.d_nodes_capacity,\n                            island.d_offsets, island.d_sizes, island.d_results, island.d_pop_capacity);\n\n    // 4. Process results\n    for (size_t k = 0; k < valid_indices.size(); ++k) {\n        int idx = valid_indices[k];\n        double sum_sq_error = raw_results[k];\n        double raw_fitness = INF;\n\n        // Check for validity\n        if (!std::isnan(sum_sq_error) && !std::isinf(sum_sq_error) && sum_sq_error < 1e300) { // 1e300 as safety threshold\n             if (USE_RMSE_FITNESS) {\n                 if (x_values.size() > 0) {\n                     double mse = sum_sq_error / x_values.size();\n                     raw_fitness = sqrt(mse);\n                 }\n             } else {\n                 raw_fitness = sum_sq_error;\n             }\n        }\n\n        if (raw_fitness >= INF/2) {\n             island.population[idx].fitness = INF;\n        } else {\n             // Complexity Penalty\n             double complexity = static_cast<double>(tree_sizes[k]); // We already have the linear size\n             double penalty = complexity * COMPLEXITY_PENALTY_FACTOR;\n             island.population[idx].fitness = raw_fitness * (1.0 + penalty);\n        }\n        island.population[idx].fitness_valid = true;\n    }\n\n#else\n    // CPU Fallback (Parallel)\n    #pragma omp parallel for schedule(dynamic)\n    for (int i = 0; i < pop_size; ++i) {\n        Individual& ind = island.population[i];\n        if (ind.tree) {\n             ind.fitness = evaluate_fitness(ind.tree, targets, x_values);\n             ind.fitness_valid = true;\n        } else {\n             ind.fitness = INF;\n             ind.fitness_valid = true;\n        }\n    }\n#endif\n}\n\n\n// --- evolve_island ---\n// (Sin cambios)\nvoid GeneticAlgorithm::evolve_island(Island& island, int current_generation) {\n    int current_pop_size = island.population.size(); if (current_pop_size == 0) return;\n    auto best_it = std::min_element(island.population.begin(), island.population.end(),\n        [](const Individual& a, const Individual& b) {\n            if (!a.tree || !a.fitness_valid) return false;\n            if (!b.tree || !b.fitness_valid) return true;\n            return a.fitness < b.fitness;\n        });\n    double current_best_fitness = INF;\n    int best_idx = -1;\n    if (best_it != island.population.end() && best_it->tree && best_it->fitness_valid) {\n        best_idx = std::distance(island.population.begin(), best_it);\n        current_best_fitness = best_it->fitness;\n    }\n    island.fitness_history.push_back(current_best_fitness);\n    if (best_idx != -1 && current_best_fitness < INF) {\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n         auto local_search_result = try_local_improvement(island.population[best_idx].tree, island.population[best_idx].fitness, targets, x_values, LOCAL_SEARCH_ATTEMPTS, d_targets, d_x_values);\n#else\n         auto local_search_result = try_local_improvement(island.population[best_idx].tree, island.population[best_idx].fitness, targets, x_values, LOCAL_SEARCH_ATTEMPTS);\n#endif\n         if (local_search_result.first && local_search_result.second < island.population[best_idx].fitness) {\n             island.population[best_idx].tree = local_search_result.first;\n             island.population[best_idx].fitness = local_search_result.second;\n             island.population[best_idx].fitness_valid = true;\n             current_best_fitness = local_search_result.second;\n         }\n    }\n    if (current_best_fitness < island.best_fitness - FITNESS_EQUALITY_TOLERANCE) {\n        island.best_fitness = current_best_fitness;\n        island.stagnation_counter = 0;\n    } else if (current_best_fitness < INF) {\n        island.stagnation_counter++;\n    }\n    island.pareto_optimizer.update(island.population, targets, x_values);\n    for(const auto& ind : island.population) {\n        if(ind.tree && ind.fitness_valid && ind.fitness < PATTERN_RECORD_FITNESS_THRESHOLD) {\n            island.pattern_memory.record_success(ind.tree, ind.fitness);\n        }\n    }\n    std::vector<Individual> next_generation;\n    next_generation.reserve(current_pop_size);\n    int elite_count = std::max(1, static_cast<int>(current_pop_size * island.params.elite_percentage));\n    if (elite_count > 0 && elite_count <= current_pop_size) {\n        std::partial_sort(island.population.begin(), island.population.begin() + elite_count, island.population.end());\n        int added_elites = 0;\n        for (int i = 0; i < elite_count && i < island.population.size(); ++i) {\n             if (island.population[i].tree && island.population[i].fitness_valid) {\n                 next_generation.emplace_back(clone_tree(island.population[i].tree));\n                 next_generation.back().fitness = island.population[i].fitness;\n                 next_generation.back().fitness_valid = true;\n                 added_elites++;\n             }\n        }\n        elite_count = added_elites;\n    } else { elite_count = 0; }\n    int random_injection_count = 0;\n    if (island.stagnation_counter > STAGNATION_LIMIT_ISLAND / 2) {\n        random_injection_count = static_cast<int>(current_pop_size * STAGNATION_RANDOM_INJECT_PERCENT);\n        for(int i = 0; i < random_injection_count && next_generation.size() < current_pop_size; ++i) {\n             NodePtr random_tree = generate_random_tree(MAX_TREE_DEPTH_INITIAL);\n             if (random_tree) next_generation.emplace_back(std::move(random_tree));\n        }\n    }\n    int pattern_injection_count = 0;\n    \n    // --- ISLAND CATACLYSM ---\n    // If enabled, triggers a hard reset if stagnation persists.\n    if (USE_ISLAND_CATACLYSM && island.stagnation_counter >= STAGNATION_LIMIT_ISLAND) {\n        // Keep only top 1 elite (already in next_generation[0] if elite_count > 0)\n        // Or if we need to enforce better elitism during cataclysm:\n        \n        int survivors = 1; // Only the absolute best one survives\n        // Resize to survivors\n        if (next_generation.size() > survivors) next_generation.resize(survivors);\n        \n        // Fill the rest with completely random trees\n        int to_fill = current_pop_size - next_generation.size();\n        for(int i=0; i<to_fill; ++i) {\n             NodePtr random_tree = generate_random_tree(MAX_TREE_DEPTH_INITIAL);\n             if (random_tree) next_generation.emplace_back(std::move(random_tree));\n        }\n        \n        island.stagnation_counter = 0; // Reset counter\n        // Optional: Pattern injection could also happen here, but random is better for total diversity.\n    }\n    // Only do standard injections if we didn't just nuke everything\n    else {\n        if (random_injection_count == 0 && current_generation % PATTERN_INJECT_INTERVAL == 0) {\n            pattern_injection_count = static_cast<int>(current_pop_size * PATTERN_INJECT_PERCENT);\n            for (int i = 0; i < pattern_injection_count && next_generation.size() < current_pop_size; ++i) {\n                NodePtr pt = island.pattern_memory.suggest_pattern_based_tree(MAX_TREE_DEPTH_INITIAL);\n                if (pt) { next_generation.emplace_back(std::move(pt)); }\n                else {\n                     NodePtr random_tree = generate_random_tree(MAX_TREE_DEPTH_INITIAL);\n                     if (random_tree) next_generation.emplace_back(std::move(random_tree));\n                }\n            }\n        }\n    }\n    auto& rng = get_rng();\n    std::uniform_real_distribution<double> prob_dist(0.0, 1.0);\n    int remaining_slots = current_pop_size - next_generation.size();\n    if (remaining_slots > 0 && !island.population.empty()) {\n         long long valid_parent_count = std::count_if(island.population.begin(), island.population.end(),\n                                             [](const Individual& ind){ return ind.tree && ind.fitness_valid; });\n         if (valid_parent_count > 0) {\n             for (int i = 0; i < remaining_slots; ++i) {\n                 const Individual& p1 = tournament_selection(island.population, island.params.tournament_size);\n                 Individual child;\n                 if (prob_dist(rng) < island.params.crossover_rate && valid_parent_count >= 2) {\n                     const Individual& p2 = tournament_selection(island.population, island.params.tournament_size);\n                     if (p1.tree && p2.tree) {\n                         NodePtr t1 = clone_tree(p1.tree); NodePtr t2 = clone_tree(p2.tree);\n                         crossover_trees(t1, t2); child.tree = t1;\n                     } else if (p1.tree) { child.tree = clone_tree(p1.tree); }\n                 } else { if (p1.tree) child.tree = clone_tree(p1.tree); }\n                 if (child.tree) child.tree = mutate_tree(child.tree, island.params.mutation_rate, MAX_TREE_DEPTH_MUTATION);\n                 child.fitness_valid = false; next_generation.push_back(std::move(child));\n             }\n         } else {\n              for (int i = 0; i < remaining_slots; ++i) {\n                   NodePtr random_tree = generate_random_tree(MAX_TREE_DEPTH_INITIAL);\n                   if (random_tree) next_generation.emplace_back(std::move(random_tree));\n              }\n         }\n    }\n    if (next_generation.size() < current_pop_size) {\n         int gap = current_pop_size - next_generation.size();\n         for (int i = 0; i < gap; ++i) {\n              NodePtr random_tree = generate_random_tree(MAX_TREE_DEPTH_INITIAL);\n              if (random_tree) next_generation.emplace_back(std::move(random_tree));\n         }\n    }\n     if (next_generation.size() > current_pop_size) next_generation.resize(current_pop_size);\n    island.population = std::move(next_generation);\n    if (current_generation > 0 && current_generation % PARAM_MUTATE_INTERVAL == 0) island.params.mutate(island.stagnation_counter);\n}\n\n// --- migrate ---\n// (Sin cambios)\nvoid GeneticAlgorithm::migrate() {\n    if (num_islands <= 1) return;\n    int current_pop_per_island = islands.empty() ? 0 : islands[0]->population.size();\n    if (current_pop_per_island == 0) return;\n    int num_migrants = std::min(MIGRATION_SIZE, current_pop_per_island / 5);\n    if (num_migrants <= 0) return;\n    std::vector<std::vector<Individual>> outgoing_migrants(num_islands);\n    #pragma omp parallel for\n    for (int i = 0; i < num_islands; ++i) {\n        Island& src = *islands[i];\n        if (src.population.size() < num_migrants) continue;\n        std::partial_sort(src.population.begin(), src.population.begin() + num_migrants, src.population.end());\n        outgoing_migrants[i].reserve(num_migrants);\n        int migrants_selected = 0;\n        for (int j = 0; j < src.population.size() && migrants_selected < num_migrants; ++j) {\n             if (src.population[j].tree && src.population[j].fitness_valid) {\n                 Individual migrant_copy;\n                 migrant_copy.tree = clone_tree(src.population[j].tree);\n                 migrant_copy.fitness = src.population[j].fitness;\n                 migrant_copy.fitness_valid = true;\n                 outgoing_migrants[i].push_back(std::move(migrant_copy));\n                 migrants_selected++;\n             }\n        }\n    }\n    for (int dest_idx = 0; dest_idx < num_islands; ++dest_idx) {\n        int src_idx = (dest_idx + num_islands - 1) % num_islands;\n        Island& dest = *islands[dest_idx];\n        const auto& migrants_to_receive = outgoing_migrants[src_idx];\n        if (migrants_to_receive.empty() || dest.population.empty()) continue;\n        int replace_count = std::min((int)migrants_to_receive.size(), (int)dest.population.size());\n        if (replace_count <= 0) continue;\n        std::partial_sort(dest.population.begin(), dest.population.end() - replace_count, dest.population.end());\n        int migrant_idx = 0;\n        for (int i = 0; i < replace_count; ++i) {\n            int replace_idx = dest.population.size() - 1 - i;\n            if (migrant_idx < migrants_to_receive.size()) {\n                 dest.population[replace_idx] = std::move(migrants_to_receive[migrant_idx++]);\n                 dest.population[replace_idx].fitness_valid = false; // Marcar para reevaluar\n            }\n        }\n    }\n}\n\n\n// --- run ---\n// (Sin cambios)\nNodePtr GeneticAlgorithm::run() {\n    std::cout << \"Starting Genetic Algorithm...\" << std::endl;\n    auto start_time = std::chrono::high_resolution_clock::now();\n\n    for (int gen = 0; gen < generations; ++gen) {\n        // 1. Evaluate Population\n        // === OPTIMIZACI\u00d3N: Paralelo en modo CPU, serial en modo GPU ===\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n        // Serial loop for GPU mode to prevent context contention\n        for (int i = 0; i < static_cast<int>(islands.size()); ++i) {\n             evaluate_population(*islands[i]);\n        }\n#else\n        // Parallel loop for CPU mode to maximize core utilization\n        #pragma omp parallel for schedule(dynamic)\n        for (int i = 0; i < static_cast<int>(islands.size()); ++i) {\n             evaluate_population(*islands[i]);\n        }\n#endif\n\n        // 2. Evolve Islands (Parallel Island Loop)\n        // Genetic operators (crossover, mutation) are CPU-bound and independent per island.\n        #pragma omp parallel for\n        for (int i = 0; i < islands.size(); ++i) {\n             evolve_island(*islands[i], gen);\n        }\n\n        double current_gen_best_fitness = INF;\n        int best_island_idx = -1;\n        int best_ind_idx = -1;\n        for (int i = 0; i < islands.size(); ++i) {\n             for (int j = 0; j < islands[i]->population.size(); ++j) {\n                 const auto& ind = islands[i]->population[j];\n                 if (ind.tree && ind.fitness_valid && ind.fitness < current_gen_best_fitness) {\n                     current_gen_best_fitness = ind.fitness;\n                     best_island_idx = i; best_ind_idx = j;\n                 }\n             }\n        }\n\n        if (best_island_idx != -1 && current_gen_best_fitness < overall_best_fitness) {\n             if (current_gen_best_fitness < overall_best_fitness) {\n                  overall_best_fitness = current_gen_best_fitness;\n                  overall_best_tree = clone_tree(islands[best_island_idx]->population[best_ind_idx].tree);\n                  std::cout << \"\\n========================================\" << std::endl;\n                  std::cout << \"New Global Best Found (Gen \" << gen + 1 << \", Island \" << best_island_idx << \")\" << std::endl;\n                  std::cout << \"Fitness: \" << std::fixed << std::setprecision(8) << overall_best_fitness << std::endl;\n                  std::cout << \"Size: \" << tree_size(overall_best_tree) << std::endl;\n                  std::cout << \"Formula: \" << tree_to_string(overall_best_tree) << std::endl;\n                  std::cout << \"Predictions vs Targets:\" << std::endl;\n                  std::cout << std::fixed << std::setprecision(4);\n                  if (overall_best_tree && !x_values.empty()) {\n                      for (size_t j = 0; j < x_values.size(); ++j) {\n                          double val = evaluate_tree(overall_best_tree, x_values[j]);\n                          double target_val = (j < targets.size()) ? targets[j] : std::nan(\"\");\n                          double diff = (!std::isnan(val) && !std::isnan(target_val)) ? std::fabs(val - target_val) : std::nan(\"\");\n                          std::cout << \"  x=\" << std::setw(8) << x_values[j]\n                                    << \": Pred=\" << std::setw(12) << val\n                                    << \", Target=\" << std::setw(12) << target_val\n                                    << \", Diff=\" << std::setw(12) << diff << std::endl;\n                      }\n                  } else { std::cout << \"  (No data or no valid tree to show predictions)\" << std::endl; }\n                  std::cout << \"========================================\" << std::endl;\n                  last_overall_best_fitness = overall_best_fitness;\n                  generation_last_improvement = gen;\n              }\n        } else {\n             if (overall_best_fitness < INF && (gen - generation_last_improvement) >= GLOBAL_STAGNATION_LIMIT) {\n                  std::cout << \"\\n========================================\" << std::endl;\n                  std::cout << \"TERMINATION: Global best fitness hasn't improved for \" << GLOBAL_STAGNATION_LIMIT << \" generations.\" << std::endl;\n                  std::cout << \"Stopping at Generation \" << gen + 1 << \".\" << std::endl;\n                  std::cout << \"========================================\" << std::endl;\n                  break;\n             }\n        }\n\n        if ((gen + 1) % MIGRATION_INTERVAL == 0 && num_islands > 1) {\n             migrate();\n             // Serial evaluation after migration\n             for (int i = 0; i < islands.size(); ++i) { evaluate_population(*islands[i]); }\n        }\n\n        if (overall_best_fitness < EXACT_SOLUTION_THRESHOLD) {\n            std::cout << \"\\n========================================\" << std::endl;\n            std::cout << \"Solution found meeting criteria at Generation \" << gen + 1 << \"!\" << std::endl;\n            std::cout << \"Final Fitness: \" << std::fixed << std::setprecision(8) << overall_best_fitness << std::endl;\n            if(overall_best_tree) {\n                 std::cout << \"Final Formula Size: \" << tree_size(overall_best_tree) << std::endl;\n                 std::cout << \"Final Formula: \" << tree_to_string(overall_best_tree) << std::endl;\n            }\n            std::cout << \"========================================\" << std::endl;\n            break;\n        }\n\n        if ((gen + 1) % PROGRESS_REPORT_INTERVAL == 0 || gen == generations - 1) {\n             auto current_time = std::chrono::high_resolution_clock::now();\n             std::chrono::duration<double> elapsed = current_time - start_time;\n             std::cout << \"\\n--- Generation \" << gen + 1 << \"/\" << generations\n                       << \" (Elapsed: \" << std::fixed << std::setprecision(2) << elapsed.count() << \"s) ---\" << std::endl;\n             std::cout << \"Overall Best Fitness: \" << std::scientific << overall_best_fitness << std::fixed << std::endl;\n              if(overall_best_tree) { std::cout << \"Best Formula Size: \" << tree_size(overall_best_tree) << std::endl; }\n              else { std::cout << \"Best Formula Size: N/A\" << std::endl; }\n              std::cout << \"(Last improvement at gen: \" << generation_last_improvement + 1 << \")\" << std::endl;\n        }\n    }\n\n    auto end_time = std::chrono::high_resolution_clock::now();\n    std::chrono::duration<double> total_elapsed = end_time - start_time;\n    std::cout << \"\\n========================================\" << std::endl;\n    std::cout << \"Evolution Finished!\" << std::endl;\n    std::cout << \"Total Time: \" << std::fixed << std::setprecision(2) << total_elapsed.count() << \" seconds\" << std::endl;\n    std::cout << \"Final Best Fitness: \" << std::fixed << std::setprecision(8) << overall_best_fitness << std::endl;\n     if (overall_best_tree) {\n         std::cout << \"Final Best Formula Size: \" << tree_size(overall_best_tree) << std::endl;\n         std::cout << \"Final Best Formula: \" << tree_to_string(overall_best_tree) << std::endl;\n          std::cout << \"--- Final Verification ---\" << std::endl;\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n          double final_check_fitness = evaluate_fitness(overall_best_tree, targets, x_values, d_targets, d_x_values);\n#else\n          double final_check_fitness = evaluate_fitness(overall_best_tree, targets, x_values);\n#endif\n          std::cout << \"Recalculated Fitness: \" << std::fixed << std::setprecision(8) << final_check_fitness << std::endl;\n          std::cout << std::fixed << std::setprecision(4);\n          for (size_t j = 0; j < x_values.size(); ++j) {\n                double val = evaluate_tree(overall_best_tree, x_values[j]);\n                 double target_val = (j < targets.size()) ? targets[j] : std::nan(\"\");\n                 double diff = (!std::isnan(val) && !std::isnan(target_val)) ? std::fabs(val - target_val) : std::nan(\"\");\n                 std::cout << \"  x=\" << std::setw(8) << x_values[j]\n                          << \": Pred=\" << std::setw(12) << val\n                          << \", Target=\" << std::setw(12) << target_val\n                          << \", Diff=\" << std::setw(12) << diff << std::endl;\n           }\n     } else { std::cout << \"No valid solution found.\" << std::endl; }\n      std::cout << \"========================================\" << std::endl;\n    return overall_best_tree;\n}\n",
  "src/GeneticAlgorithm.h": "// ============================================================\n// Archivo: src/GeneticAlgorithm.h\n// ============================================================\n#ifndef GENETICALGORITHM_H\n#define GENETICALGORITHM_H\n\n#include \"ExpressionTree.h\"\n#include \"GeneticOperators.h\"\n#include \"AdvancedFeatures.h\"\n#include \"Globals.h\" // Incluir Globals.h para INF, NUM_ISLANDS, etc.\n#include <vector>\n#include <string>\n#include <memory> // Para std::unique_ptr\n\nclass GeneticAlgorithm {\n    // Estructura interna para representar una isla\n    struct Island {\n        std::vector<Individual> population; // Poblaci\u00f3n de la isla\n        EvolutionParameters params;         // Par\u00e1metros evolutivos propios de la isla\n        PatternMemory pattern_memory;       // Memoria de patrones de la isla\n        ParetoOptimizer pareto_optimizer;   // Optimizador Pareto de la isla\n        int stagnation_counter = 0;         // Contador de estancamiento local de la isla\n        double best_fitness = INF;          // Mejor fitness hist\u00f3rico de la isla\n        std::vector<double> fitness_history;// Historial de fitness (opcional)\n        int id;                             // Identificador de la isla\n\n        // Constructor de la isla\n        explicit Island(int island_id, int pop_size) : id(island_id) {\n             population = create_initial_population(pop_size); // Crear poblaci\u00f3n inicial\n             params = EvolutionParameters::create_default();   // Usar par\u00e1metros por defecto\n        }\n\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n        // Persistent GPU buffers\n        void* d_nodes = nullptr;\n        void* d_offsets = nullptr;\n        void* d_sizes = nullptr;\n        void* d_results = nullptr;\n        size_t d_nodes_capacity = 0;\n        size_t d_pop_capacity = 0;\n\n        ~Island() {\n            // We cannot easily call cudaFree here because this header might be included\n            // where cuda_runtime.h is not. However, we can trust the OS/driver to clean up\n            // or we should add a cleanup function.\n            // For now, we will rely on GeneticAlgorithm destructor or explict cleanup if possible.\n            // But since Island is unique_ptr, we can't easily add a destructor that calls cudaFree \n            // without including cuda_runtime.\n            // Optimization: Let's rely on OS cleanup at exit, OR add a cleanup method called by GA.\n        }\n#endif\n    };\n\n    // Miembros principales de la clase GeneticAlgorithm\n    std::vector<std::unique_ptr<Island>> islands; // Vector de punteros \u00fanicos a las islas\n    const std::vector<double>& targets;           // Referencia a los datos objetivo\n    const std::vector<double>& x_values;          // Referencia a los valores de x\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n    double* d_targets = nullptr;                  // Puntero a los datos objetivo en la GPU\n    double* d_x_values = nullptr;                 // Puntero a los valores de x en la GPU\n#endif\n    int total_population_size;                    // Tama\u00f1o total de la poblaci\u00f3n\n    int generations;                              // N\u00famero m\u00e1ximo de generaciones\n    int num_islands;                              // N\u00famero de islas\n\n    // Seguimiento del mejor global\n    NodePtr overall_best_tree = nullptr;          // Mejor \u00e1rbol encontrado globalmente\n    double overall_best_fitness = INF;            // Mejor fitness encontrado globalmente\n\n    // --- NUEVO: Seguimiento de Estancamiento Global ---\n    int generation_last_improvement = 0;          // Generaci\u00f3n en la que mejor\u00f3 el overall_best_fitness\n    double last_overall_best_fitness = INF;       // Valor del overall_best_fitness en la \u00faltima mejora\n    // -------------------------------------------------\n\n    int pop_per_island;                           // Poblaci\u00f3n calculada por isla\n\npublic:\n    // Constructor\n    GeneticAlgorithm(const std::vector<double>& targets_ref,\n                       const std::vector<double>& x_values_ref,\n                       int total_pop,\n                       int gens,\n                       int n_islands = NUM_ISLANDS); // Usar valor de Globals.h por defecto\n    ~GeneticAlgorithm(); // Destructor para liberar memoria de la GPU\n\n    // Ejecuta el algoritmo gen\u00e9tico\n    NodePtr run();\n\nprivate:\n    // Funciones auxiliares internas\n    void evaluate_population(Island& island); // Eval\u00faa fitness de una isla\n    void evolve_island(Island& island, int current_generation); // Evoluciona una isla por una generaci\u00f3n\n    void migrate(); // Realiza la migraci\u00f3n entre islas\n    void update_overall_best(const Island& island); // Actualiza el mejor global\n};\n\n\n#endif // GENETICALGORITHM_H\n",
  "src/GeneticOperators.cpp": "#include \"GeneticOperators.h\"\n#include \"Globals.h\"\n#include \"Fitness.h\"\n#include \"AdvancedFeatures.h\"\n#include <vector>\n#include <cmath>\n#include <algorithm>\n#include <map>\n#include <stdexcept>\n#include <set>\n#include <iostream> // Para mensajes de error/info\n\n// Genera un \u00e1rbol aleatorio (CON TODOS LOS OPERADORES, EXPONENTES SIN RESTRICCI\u00d3N)\nNodePtr generate_random_tree(int max_depth, int current_depth) {\n    std::uniform_real_distribution<double> prob_dist(0.0, 1.0);\n    auto& rng = get_rng();\n    double terminal_prob = 0.2 + 0.8 * (static_cast<double>(current_depth) / max_depth);\n\n    if (current_depth >= max_depth || prob_dist(rng) < terminal_prob) {\n        // Crear terminal\n        if (prob_dist(rng) < TERMINAL_VS_VARIABLE_PROB) { return std::make_shared<Node>(NodeType::Variable); }\n        else {\n            auto node = std::make_shared<Node>(NodeType::Constant);\n            if (FORCE_INTEGER_CONSTANTS) { std::uniform_int_distribution<int> cd(CONSTANT_INT_MIN_VALUE, CONSTANT_INT_MAX_VALUE); node->value = static_cast<double>(cd(rng)); }\n            else { std::uniform_real_distribution<double> cd(CONSTANT_MIN_VALUE, CONSTANT_MAX_VALUE); node->value = cd(rng); }\n            if (std::fabs(node->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) node->value = 0.0;\n            return node;\n        }\n    } else {\n\n        // Crear operador\n        auto node = std::make_shared<Node>(NodeType::Operator);\n        // Match the weights in Globals.h: +, -, *, /, ^, %, s, c, l, e, !, _, g\n        const std::vector<char> ops = {'+', '-', '*', '/', '^', '%', 's', 'c', 'l', 'e', '!', '_', 'g'};\n        std::discrete_distribution<int> op_dist(OPERATOR_WEIGHTS.begin(), OPERATOR_WEIGHTS.end());\n        node->op = ops[op_dist(rng)];\n\n        bool is_unary = (node->op == 's' || node->op == 'c' || node->op == 'l' || node->op == 'e' || node->op == '!' || node->op == '_' || node->op == 'g');\n\n        // Generar hijos recursivamente\n        node->left = generate_random_tree(max_depth, current_depth + 1);\n        if (!is_unary) {\n            node->right = generate_random_tree(max_depth, current_depth + 1);\n        } else {\n            node->right = nullptr;\n        }\n\n        // Fallback para hijos nulos\n        auto generate_random_terminal = [&]() -> NodePtr {\n            if (prob_dist(rng) < TERMINAL_VS_VARIABLE_PROB) { return std::make_shared<Node>(NodeType::Variable); }\n            else {\n                auto const_node = std::make_shared<Node>(NodeType::Constant);\n                if (FORCE_INTEGER_CONSTANTS) { std::uniform_int_distribution<int> cd(CONSTANT_INT_MIN_VALUE, CONSTANT_INT_MAX_VALUE); const_node->value = static_cast<double>(cd(rng)); }\n                else { std::uniform_real_distribution<double> cd(CONSTANT_MIN_VALUE, CONSTANT_MAX_VALUE); const_node->value = cd(rng); }\n                if (std::fabs(const_node->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) const_node->value = 0.0;\n                return const_node;\n            }\n        };\n\n        if (!node->left) node->left = generate_random_terminal();\n        if (!is_unary && !node->right) node->right = generate_random_terminal();\n\n        // --- Manejo especial para el operador de potencia '^' ---\n        if (node->op == '^') {\n            // Regla 1: Evitar 0^0 o 0^negativo\n            if (node->left->type == NodeType::Constant && std::fabs(node->left->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) {\n                if (node->right->type == NodeType::Constant && node->right->value <= SIMPLIFY_NEAR_ZERO_TOLERANCE) {\n                    const std::vector<char> safe_ops = {'+', '-', '*'};\n                    std::uniform_int_distribution<int> safe_op_dist(0, safe_ops.size() - 1);\n                    node->op = safe_ops[safe_op_dist(rng)];\n                }\n            }\n            // Regla 2: Evitar base negativa con exponente no entero\n            else if (node->left->type == NodeType::Constant && node->left->value < 0.0) {\n                if (node->right->type == NodeType::Constant && std::fabs(node->right->value - std::round(node->right->value)) > SIMPLIFY_NEAR_ZERO_TOLERANCE) {\n                     // Change exponent to int\n                     std::uniform_int_distribution<int> int_exp_dist(-3, 3);\n                     node->right = std::make_shared<Node>(NodeType::Constant);\n                     node->right->value = static_cast<double>(int_exp_dist(rng));\n                }\n            }\n        }\n        return node;\n    }\n}\n\n// --- Crea la poblaci\u00f3n inicial (MODIFICADO para inyectar f\u00f3rmula) ---\n// === OPTIMIZACI\u00d3N: Paralelizado con OpenMP ===\nstd::vector<Individual> create_initial_population(int population_size) {\n    std::vector<Individual> population;\n    population.resize(population_size); // Pre-allocate all slots for parallel access\n\n    // --- NUEVO: Inyecci\u00f3n de F\u00f3rmula Inicial ---\n    int start_index = 0;\n    if (USE_INITIAL_FORMULA && !INITIAL_FORMULA_STRING.empty() && population_size > 0) {\n        try {\n            NodePtr initial_tree = parse_formula_string(INITIAL_FORMULA_STRING);\n            if (initial_tree) {\n                population[0] = Individual(std::move(initial_tree));\n                start_index = 1;\n                std::cout << \"[INFO] Injected initial formula: \" << INITIAL_FORMULA_STRING << std::endl;\n            } else {\n                 std::cerr << \"[WARNING] Parsing initial formula returned null. Skipping injection.\" << std::endl;\n            }\n        } catch (const std::exception& e) {\n            std::cerr << \"[ERROR] Failed to parse initial formula '\" << INITIAL_FORMULA_STRING\n                      << \"': \" << e.what() << \". Skipping injection.\" << std::endl;\n        }\n    }\n    // -----------------------------------------\n\n    // === OPTIMIZACI\u00d3N: Loop paralelo para generar \u00e1rboles ===\n    #pragma omp parallel for schedule(dynamic, 100)\n    for (int i = start_index; i < population_size; ++i) {\n        // Cada hilo tiene su propio RNG (thread_local en get_rng)\n        auto& rng = get_rng();\n        std::uniform_int_distribution<int> depth_dist(3, MAX_TREE_DEPTH_INITIAL);\n        \n        NodePtr random_tree = nullptr;\n        int attempts = 0;\n        const int max_attempts = 10;\n        while (!random_tree && attempts < max_attempts) {\n            random_tree = generate_random_tree(depth_dist(rng));\n            attempts++;\n        }\n        if (random_tree) {\n            population[i] = Individual(std::move(random_tree));\n        } else {\n            // Fallback: crear constante simple\n            auto fallback_node = std::make_shared<Node>(NodeType::Constant);\n            fallback_node->value = 0.0;\n            population[i] = Individual(std::move(fallback_node));\n        }\n    }\n    return population;\n}\n\n// --- Selecci\u00f3n por torneo con parsimonia ---\nIndividual tournament_selection(const std::vector<Individual>& population, int tournament_size) {\n    if (population.empty()) throw std::runtime_error(\"Cannot perform tournament selection on empty population.\");\n    if (tournament_size <= 0) tournament_size = 1;\n    tournament_size = std::min(tournament_size, (int)population.size());\n\n    std::uniform_int_distribution<int> dist(0, population.size() - 1);\n    auto& rng = get_rng();\n    const Individual* best_in_tournament = nullptr;\n\n    int attempts = 0; const int max_attempts = std::min((int)population.size() * 2, 100);\n    do {\n        best_in_tournament = &population[dist(rng)];\n        attempts++;\n    } while ((!best_in_tournament || !best_in_tournament->tree || !best_in_tournament->fitness_valid) && attempts < max_attempts);\n\n    if (!best_in_tournament || !best_in_tournament->tree || !best_in_tournament->fitness_valid) {\n         if (!population.empty()) return population[0];\n         else throw std::runtime_error(\"Tournament selection couldn't find any valid individual in a non-empty population.\");\n    }\n\n    for (int i = 1; i < tournament_size; ++i) {\n        const Individual& contender = population[dist(rng)];\n        if (!contender.tree || !contender.fitness_valid) continue;\n\n        if (contender.fitness < best_in_tournament->fitness) {\n            best_in_tournament = &contender;\n        }\n        else if (std::fabs(contender.fitness - best_in_tournament->fitness) < FITNESS_EQUALITY_TOLERANCE) {\n            int contender_size = tree_size(contender.tree);\n            int best_size = tree_size(best_in_tournament->tree);\n            if (contender_size < best_size) best_in_tournament = &contender;\n        }\n    }\n    return *best_in_tournament;\n}\n\n// Implementaci\u00f3n de crossover\nIndividual crossover(const Individual& parent1, const Individual& parent2) {\n    NodePtr tree1_clone = clone_tree(parent1.tree);\n    NodePtr tree2_clone = clone_tree(parent2.tree);\n    crossover_trees(tree1_clone, tree2_clone);\n    return Individual(tree1_clone); // Devolver uno de los hijos, el otro se descarta\n}\n\n// Implementaci\u00f3n de mutate\nvoid mutate(Individual& individual, double mutation_rate) {\n    individual.tree = mutate_tree(individual.tree, mutation_rate, MAX_TREE_DEPTH_MUTATION);\n    individual.fitness_valid = false; // El fitness se invalida al mutar el \u00e1rbol\n}\n\n// Mutata un \u00e1rbol (EXPONENTES SIN RESTRICCI\u00d3N en OperatorChange)\nNodePtr mutate_tree(const NodePtr& tree, double mutation_rate, int max_depth) {\n    auto& rng = get_rng();\n    std::uniform_real_distribution<double> prob(0.0, 1.0);\n    auto new_tree = clone_tree(tree); // Siempre clonar primero\n    if (!new_tree) return nullptr; // Si el \u00e1rbol original era nulo, el clon tambi\u00e9n\n\n    if (prob(rng) >= mutation_rate) return new_tree; // No mutar\n\n    std::vector<NodePtr*> nodes; collect_node_ptrs(new_tree, nodes);\n    if (nodes.empty()) return new_tree; // No hay nodos para mutar (\u00e1rbol vac\u00edo?)\n\n    std::uniform_int_distribution<int> node_dist(0, nodes.size() - 1);\n    int node_idx = node_dist(rng);\n    NodePtr* node_to_mutate_ptr = nodes[node_idx];\n    if (!node_to_mutate_ptr || !(*node_to_mutate_ptr)) return new_tree; // Puntero o nodo nulo inesperado\n\n    const std::vector<MutationType> mutation_types = {\n        MutationType::ConstantChange, MutationType::OperatorChange,\n        MutationType::SubtreeReplace, MutationType::NodeInsertion,\n        MutationType::NodeDeletion\n    };\n    std::uniform_int_distribution<int> type_dist(0, mutation_types.size() - 1);\n    MutationType mut_type = mutation_types[type_dist(rng)];\n\n    NodePtr& current_node_ptr_ref = *node_to_mutate_ptr;\n    Node& current_node = *current_node_ptr_ref;\n\n    // Generar reemplazo aleatorio (usado en varios casos)\n    auto generate_replacement = [&](int depth) -> NodePtr {\n        NodePtr replacement = generate_random_tree(depth);\n        if (!replacement) { // Fallback si la generaci\u00f3n falla\n            replacement = std::make_shared<Node>(NodeType::Constant);\n            replacement->value = 1.0; // Usar 1.0 como fallback simple\n        }\n        return replacement;\n    };\n\n\n    switch (mut_type) {\n        case MutationType::ConstantChange:\n             if (current_node.type == NodeType::Constant) {\n                 // Cambiar valor de la constante\n                 double change_factor = std::uniform_real_distribution<double>(0.8, 1.2)(rng);\n                 double add_factor = std::uniform_real_distribution<double>(-1.0, 1.0)(rng);\n                 current_node.value = current_node.value * change_factor + add_factor;\n                 if (FORCE_INTEGER_CONSTANTS) current_node.value = std::round(current_node.value);\n                 if (std::fabs(current_node.value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) current_node.value = 0.0;\n             } else {\n                 // Si no es constante, reemplazar por un sub\u00e1rbol aleatorio peque\u00f1o\n                 *node_to_mutate_ptr = generate_replacement(1); // Profundidad 1 (terminal)\n             }\n            break;\n        case MutationType::OperatorChange:\n             if (current_node.type == NodeType::Operator) {\n                 // Usar la distribuci\u00f3n ponderada global para elegir el nuevo operador.\n                 // Esto asegura que si 'g' tiene peso alto, sea elegido frecuentemente.\n                 // Asumimos que OPERATOR_WEIGHTS tiene 0.0 para operadores deshabilitados.\n                 \n                 const std::vector<char> all_ops = {'+', '-', '*', '/', '^', '%', 's', 'c', 'l', 'e', '!', '_', 'g'};\n                 std::discrete_distribution<int> op_dist(OPERATOR_WEIGHTS.begin(), OPERATOR_WEIGHTS.end());\n                 \n                 // Verificar si hay al menos 2 operadores habilitados para evitar bucle infinito\n                 int enabled_count = 0;\n                 for (double w : OPERATOR_WEIGHTS) if (w > 0.0) enabled_count++;\n                 \n                 if (enabled_count > 1) {\n                     int attempts = 0;\n                     int new_op_idx = -1;\n                     do {\n                         new_op_idx = op_dist(rng);\n                         attempts++;\n                     } while (all_ops[new_op_idx] == current_node.op && attempts < 20); // Intentar cambiar\n                     \n                     if (attempts < 20) { // Si logramos encontrar uno diferente\n                         char old_op = current_node.op;\n                         char new_op = all_ops[new_op_idx];\n                         \n                         bool was_unary = (old_op == 's' || old_op == 'c' || old_op == 'l' || old_op == 'e' || old_op == '!' || old_op == '_' || old_op == 'g');\n                         bool is_unary = (new_op == 's' || new_op == 'c' || new_op == 'l' || new_op == 'e' || new_op == '!' || new_op == '_' || new_op == 'g');\n\n                         if (was_unary && !is_unary) {\n                             current_node.right = generate_replacement(1);\n                         } else if (!was_unary && is_unary) {\n                             current_node.right = nullptr;\n                         }\n                         current_node.op = new_op;\n                     }\n                 }\n             } else {\n                 // Si no es operador, reemplazar por un sub\u00e1rbol aleatorio\n                  *node_to_mutate_ptr = generate_replacement(max_depth);\n             }\n            break;\n        case MutationType::SubtreeReplace:\n            *node_to_mutate_ptr = generate_replacement(max_depth);\n            break;\n        case MutationType::NodeInsertion:\n            {\n                auto new_op_node = std::make_shared<Node>(NodeType::Operator);\n                // Usar distribuci\u00f3n ponderada\n                const std::vector<char> all_ops = {'+', '-', '*', '/', '^', '%', 's', 'c', 'l', 'e', '!', '_', 'g'};\n                std::discrete_distribution<int> op_dist(OPERATOR_WEIGHTS.begin(), OPERATOR_WEIGHTS.end());\n                \n                int new_op_idx = op_dist(rng); // Siempre elegir\u00e1 uno habilitado\n                new_op_node->op = all_ops[new_op_idx];\n\n                bool is_unary = (new_op_node->op == 's' || new_op_node->op == 'c' || new_op_node->op == 'l' || new_op_node->op == 'e' || new_op_node->op == '!' || new_op_node->op == '_' || new_op_node->op == 'g');\n\n                new_op_node->left = current_node_ptr_ref;\n\n                if (!is_unary) {\n                     if (prob(rng) < MUTATE_INSERT_CONST_PROB) {\n                         auto right_child = std::make_shared<Node>(NodeType::Constant);\n                         if (FORCE_INTEGER_CONSTANTS) { std::uniform_int_distribution<int> cv(MUTATE_INSERT_CONST_INT_MIN, MUTATE_INSERT_CONST_INT_MAX); right_child->value = static_cast<double>(cv(rng)); }\n                         else { std::uniform_real_distribution<double> cv(MUTATE_INSERT_CONST_FLOAT_MIN, MUTATE_INSERT_CONST_FLOAT_MAX); right_child->value = cv(rng); }\n                         if (std::fabs(right_child->value) < SIMPLIFY_NEAR_ZERO_TOLERANCE) right_child->value = 0.0;\n                         new_op_node->right = right_child;\n                     } else {\n                         new_op_node->right = std::make_shared<Node>(NodeType::Variable);\n                     }\n                     if (!new_op_node->right) new_op_node->right = std::make_shared<Node>(NodeType::Variable);\n                } else {\n                    new_op_node->right = nullptr;\n                }\n\n                *node_to_mutate_ptr = new_op_node;\n            }\n            break;\n        case MutationType::NodeDeletion:\n            {\n                // No eliminar la ra\u00edz directamente si es la \u00fanica opci\u00f3n\n                if (node_to_mutate_ptr == &new_tree && nodes.size() == 1) return new_tree;\n\n                if (current_node.type == NodeType::Operator) {\n                    // Si es operador, reemplazarlo por uno de sus hijos (aleatorio)\n                    NodePtr replacement = nullptr;\n                    bool has_left = (current_node.left != nullptr);\n                    bool has_right = (current_node.right != nullptr);\n\n                    if (has_left && has_right) {\n                        replacement = (prob(rng) < 0.5) ? current_node.left : current_node.right;\n                    } else if (has_left) {\n                        replacement = current_node.left;\n                    } else if (has_right) {\n                        replacement = current_node.right;\n                    }\n                    // Si no tiene hijos v\u00e1lidos (\u00bfc\u00f3mo?), reemplazar por terminal\n                    if (!replacement) replacement = generate_replacement(0); // Profundidad 0 (terminal)\n\n                    *node_to_mutate_ptr = replacement;\n                } else {\n                    // Si es terminal, reemplazar por otro terminal aleatorio\n                    // (Evitar eliminar si es la ra\u00edz y no hay m\u00e1s nodos)\n                     if (node_to_mutate_ptr != &new_tree || nodes.size() > 1) {\n                          *node_to_mutate_ptr = generate_replacement(0); // Profundidad 0 (terminal)\n                     }\n                }\n            }\n            break;\n         default: // Caso inesperado, reemplazar por seguridad\n             *node_to_mutate_ptr = generate_replacement(max_depth);\n            break;\n    }\n    return new_tree;\n}\n\n// Cruce\nvoid crossover_trees(NodePtr& tree1, NodePtr& tree2) {\n    if (!tree1 || !tree2) return; // No cruzar si alguno es nulo\n\n    std::vector<NodePtr*> nodes1, nodes2;\n    collect_node_ptrs(tree1, nodes1);\n    collect_node_ptrs(tree2, nodes2);\n\n    // No cruzar si alguno no tiene nodos (\u00e1rbol vac\u00edo o solo ra\u00edz nula?)\n    if (nodes1.empty() || nodes2.empty()) return;\n\n    auto& rng = get_rng();\n    std::uniform_int_distribution<int> d1(0, nodes1.size()-1);\n    std::uniform_int_distribution<int> d2(0, nodes2.size()-1);\n\n    // Seleccionar puntos de cruce\n    NodePtr* crossover_point1 = nodes1[d1(rng)];\n    NodePtr* crossover_point2 = nodes2[d2(rng)];\n\n    // Intercambiar los sub\u00e1rboles (los NodePtr)\n    std::swap(*crossover_point1, *crossover_point2);\n}\n\n// Implementaci\u00f3n de simplify_tree\nvoid simplify_tree(NodePtr& tree) {\n    if (USE_SIMPLIFICATION) {\n        tree = DomainConstraints::fix_or_simplify(tree);\n    }\n}\n",
  "src/GeneticOperators.h": "// ============================================================\n// Archivo: src/GeneticOperators.h\n// ============================================================\n#ifndef GENETICOPERATORS_H\n#define GENETICOPERATORS_H\n\n#include \"ExpressionTree.h\"\n#include \"Globals.h\" // Incluir Globals.h para INF\n#include <vector>\n#include <memory> // Para std::move\n\n// Estructura para representar un individuo en la poblaci\u00f3n.\n// Contiene el \u00e1rbol de expresi\u00f3n y su fitness cacheado.\nstruct Individual {\n    NodePtr tree; // Puntero inteligente al \u00e1rbol de expresi\u00f3n\n    double fitness = INF; // Fitness cacheado (menor es mejor), inicializado a infinito\n    bool fitness_valid = false; // Indica si el fitness cacheado es v\u00e1lido\n\n    // Constructor por defecto\n    Individual() = default;\n    // Constructor a partir de un \u00e1rbol (mueve el puntero)\n    explicit Individual(NodePtr t) : tree(std::move(t)) {}\n\n    // Operador de comparaci\u00f3n para ordenar individuos (menor fitness primero)\n    bool operator<(const Individual& other) const {\n        // Manejar casos donde uno o ambos fitness no son v\u00e1lidos\n        if (!fitness_valid && !other.fitness_valid) return false; // Iguales si ambos inv\u00e1lidos\n        if (!fitness_valid) return false; // Inv\u00e1lido es \"peor\" que v\u00e1lido (va despu\u00e9s)\n        if (!other.fitness_valid) return true; // V\u00e1lido es \"mejor\" que inv\u00e1lido (va antes)\n        // Comparar por fitness si ambos son v\u00e1lidos\n        return fitness < other.fitness;\n    }\n};\n\n\n// --- Funciones de Operadores Gen\u00e9ticos ---\n\n// Genera un \u00e1rbol de expresi\u00f3n aleatorio hasta una profundidad m\u00e1xima.\nNodePtr generate_random_tree(int max_depth, int current_depth = 0);\n\n// Crea la poblaci\u00f3n inicial de individuos.\nstd::vector<Individual> create_initial_population(int population_size);\n\n// Selecciona un individuo usando selecci\u00f3n por torneo con presi\u00f3n de parsimonia.\nIndividual tournament_selection(const std::vector<Individual>& population, int tournament_size);\n\n// Realiza el cruce (crossover) entre dos individuos y devuelve un nuevo individuo.\nIndividual crossover(const Individual& parent1, const Individual& parent2);\n\n// Mutata un individuo in-place.\nvoid mutate(Individual& individual, double mutation_rate);\n\n// Simplifica un \u00e1rbol in-place.\nvoid simplify_tree(NodePtr& tree);\n\n// Tipos de mutaci\u00f3n posibles.\nenum class MutationType {\n    ConstantChange,\n    OperatorChange,\n    SubtreeReplace,\n    NodeInsertion,\n    NodeDeletion // <-- A\u00d1ADIDO: Tipo para eliminar un nodo\n    // Simplification (manejado por DomainConstraints)\n};\n\n// Mutata un \u00e1rbol aplicando uno de los tipos de mutaci\u00f3n con cierta probabilidad.\n// Devuelve un nuevo \u00e1rbol (clonado y potencialmente mutado).\nNodePtr mutate_tree(const NodePtr& tree, double mutation_rate, int max_depth);\n\n// Realiza el cruce (crossover) entre dos \u00e1rboles padres, modific\u00e1ndolos in-place.\nvoid crossover_trees(NodePtr& tree1, NodePtr& tree2);\n\n\n#endif // GENETICOPERATORS_H\n",
  "src/main.cpp": "#include \"Globals.h\" // Necesario para las constantes globales\n#include \"GeneticAlgorithm.h\"\n#include \"ExpressionTree.h\" // Para tree_to_string si se necesita aqu\u00ed\n#include <iostream>\n#include <vector>\n#include <memory> // Para shared_ptr\n#include <iomanip> // Para std::setprecision\n#include <omp.h>   // Para configuraci\u00f3n de OpenMP\n\nint main() {\n    // === OPTIMIZACI\u00d3N: Configuraci\u00f3n expl\u00edcita de hilos OpenMP ===\n    int num_threads = omp_get_max_threads();\n    omp_set_num_threads(num_threads);\n    std::cout << \"[OpenMP] Using \" << num_threads << \" threads\" << std::endl;\n    \n    // Configurar precisi\u00f3n de salida para n\u00fameros flotantes\n    std::cout << std::fixed << std::setprecision(6);\n\n    std::cout << \"Symbolic Regression using Genetic Programming (Island Model)\" << std::endl;\n    std::cout << \"==========================================================\" << std::endl;\n    std::vector<double> targets;\n    std::vector<double> final_x_values;\n\n    if (USE_LOG_TRANSFORMATION) {\n        std::cout << \"Info: Log Transformation is ON (Target = ln(Q(N))).\" << std::endl;\n        for (size_t i = 0; i < RAW_TARGETS.size(); ++i) {\n            if (RAW_TARGETS[i] > 0) {\n                targets.push_back(std::log(RAW_TARGETS[i]));\n                final_x_values.push_back(X_VALUES[i]);\n            }\n        }\n    } else {\n        std::cout << \"Info: Log Transformation is OFF.\" << std::endl;\n        targets = RAW_TARGETS;\n        final_x_values = X_VALUES;\n    }\n\n    std::cout << \"Target Function Points (Effective):\" << std::endl;\n    // Imprimir los puntos objetivo\n    for (size_t i = 0; i < targets.size(); ++i) {\n        std::cout << \"  f(\" << final_x_values[i] << \") = \" << targets[i] << std::endl;\n    }\n    std::cout << \"----------------------------------------\" << std::endl;\n#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE\n    std::cout << \"Info: Running with GPU acceleration.\" << std::endl;\n#else\n    std::cout << \"Info: Running with CPU acceleration.\" << std::endl;\n#endif\n    std::cout << \"----------------------------------------\" << std::endl;\n    std::cout << \"Parameters:\" << std::endl;\n    // Imprimir los par\u00e1metros globales definidos en Globals.h\n    std::cout << \"  Total Population: \" << TOTAL_POPULATION_SIZE << std::endl;\n    std::cout << \"  Generations: \" << GENERATIONS << std::endl;\n    std::cout << \"  Islands: \" << NUM_ISLANDS << std::endl;\n    std::cout << \"  Migration Interval: \" << MIGRATION_INTERVAL << std::endl;\n    std::cout << \"  Migration Size: \" << MIGRATION_SIZE << std::endl;\n    // --- NOMBRES CORREGIDOS ---\n    std::cout << \"  Mutation Rate (Initial): \" << BASE_MUTATION_RATE << std::endl; // <-- Nombre corregido\n    std::cout << \"  Elite Percentage (Initial): \" << BASE_ELITE_PERCENTAGE << std::endl; // <-- Nombre corregido\n    // --------------------------\n    std::cout << \"----------------------------------------\" << std::endl;\n\n\n    // Crear la instancia del Algoritmo Gen\u00e9tico\n    // Pasa las referencias a los vectores de datos y los par\u00e1metros principales\n    GeneticAlgorithm ga(targets, final_x_values, TOTAL_POPULATION_SIZE, GENERATIONS, NUM_ISLANDS);\n\n    // Ejecutar el algoritmo\n    // La funci\u00f3n run() contiene el bucle principal de generaciones y devuelve el mejor \u00e1rbol encontrado\n    NodePtr best_solution_tree = ga.run();\n\n    // La funci\u00f3n run() ya imprime el resumen final y la verificaci\u00f3n.\n    // Comprobar si se encontr\u00f3 alguna soluci\u00f3n v\u00e1lida al final\n    if (!best_solution_tree) {\n        std::cerr << \"\\nFailed to find any valid solution.\" << std::endl;\n        return 1; // Salir con c\u00f3digo de error si no se encontr\u00f3 soluci\u00f3n\n    }\n\n    return 0; // Salir con \u00e9xito\n}\n"
}

for filepath, content in files_to_create.items():
    os.makedirs(os.path.dirname(filepath), exist_ok=True) if os.path.dirname(filepath) else None
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f'Created: {filepath}')

## Configuration & Compilation

In [ ]:
# @title Algorithm Parameters
# @markdown Modify these values to tune the genetic algorithm.

import os

# Problem Data
RAW_TARGETS_STR = "2, 10, 4, 40, 92, 352, 724, 2680, 14200, 73712, 365596, 2279184, 14772512, 95815104, 666090624, 4968057848, 39029188884" # @param {type:"string"}
X_VALUES_STR = "4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20" # @param {type:"string"}
USE_LOG_TRANSFORMATION = True # @param {type:"boolean"}

# Algorithm Settings
USE_GPU_ACCELERATION = True # @param {type:"boolean"}
TOTAL_POPULATION_SIZE = 50000 # @param {type:"integer"}
GENERATIONS = 500000 # @param {type:"integer"}
NUM_ISLANDS = 10 # @param {type:"integer"}
MIGRATION_INTERVAL = 100 # @param {type:"integer"}
MIGRATION_SIZE = 50 # @param {type:"integer"}

# Genetic Operators
BASE_MUTATION_RATE = 0.30 # @param {type:"number"}
BASE_ELITE_PERCENTAGE = 0.15 # @param {type:"number"}

# Operator Selection
USE_OP_PLUS = True # @param {type:"boolean"}
USE_OP_MINUS = True # @param {type:"boolean"}
USE_OP_MULT = True # @param {type:"boolean"}
USE_OP_DIV = True # @param {type:"boolean"}
USE_OP_POW = True # @param {type:"boolean"}
USE_OP_MOD = True # @param {type:"boolean"}
USE_OP_SIN = True # @param {type:"boolean"}
USE_OP_COS = True # @param {type:"boolean"}
USE_OP_LOG = True # @param {type:"boolean"}
USE_OP_EXP = True # @param {type:"boolean"}
USE_OP_FACT = True # @param {type:"boolean"}
USE_OP_FLOOR = True # @param {type:"boolean"}
USE_OP_GAMMA = True # @param {type:"boolean"}

# Initial Formula Injection
USE_INITIAL_FORMULA = True # @param {type:"boolean"}
INITIAL_FORMULA_STRING = "l(g(x+1))-(x*0.92)" # @param {type:"string"}

# Optimization Settings
FORCE_INTEGER_CONSTANTS = False # @param {type:"boolean"}
USE_SIMPLIFICATION = True # @param {type:"boolean"}
USE_ISLAND_CATACLYSM = True # @param {type:"boolean"}

# Weighted Fitness
USE_WEIGHTED_FITNESS = True # @param {type:"boolean"}
WEIGHTED_FITNESS_EXPONENT = 0.25 # @param {type:"number"}

# Construct Globals.h content
globals_content = f"""
#ifndef GLOBALS_H
#define GLOBALS_H

#include <vector>
#include <random>
#include <string>
#include <limits>
#include <cmath>

// Data
const std::vector<double> RAW_TARGETS = {{ {RAW_TARGETS_STR} }};
const std::vector<double> X_VALUES = {{ {X_VALUES_STR} }};
const bool USE_LOG_TRANSFORMATION = {'true' if USE_LOG_TRANSFORMATION else 'false'};

// GPU Settings
#ifdef USE_GPU_ACCELERATION_DEFINED_BY_CMAKE
const bool USE_GPU_ACCELERATION = {'true' if USE_GPU_ACCELERATION else 'false'};
#else
const bool USE_GPU_ACCELERATION = false;
#endif

// Algorithm Parameters
const int TOTAL_POPULATION_SIZE = {TOTAL_POPULATION_SIZE};
const int GENERATIONS = {GENERATIONS};
const int NUM_ISLANDS = {NUM_ISLANDS};
const int MIN_POP_PER_ISLAND = 10;

// Migration
const int MIGRATION_INTERVAL = {MIGRATION_INTERVAL};
const int MIGRATION_SIZE = {MIGRATION_SIZE};

// Initial Formula
const bool USE_INITIAL_FORMULA = {'true' if USE_INITIAL_FORMULA else 'false'};
const std::string INITIAL_FORMULA_STRING = "{INITIAL_FORMULA_STRING}";

// Genetic Operators
const double BASE_MUTATION_RATE = {BASE_MUTATION_RATE};
const double BASE_ELITE_PERCENTAGE = {BASE_ELITE_PERCENTAGE};
const double DEFAULT_CROSSOVER_RATE = 0.85;
const int DEFAULT_TOURNAMENT_SIZE = 30;
const int MAX_TREE_DEPTH_MUTATION = 8;

// Operator Configuration
const bool USE_OP_PLUS     = {'true' if USE_OP_PLUS else 'false'};
const bool USE_OP_MINUS    = {'true' if USE_OP_MINUS else 'false'};
const bool USE_OP_MULT     = {'true' if USE_OP_MULT else 'false'};
const bool USE_OP_DIV      = {'true' if USE_OP_DIV else 'false'};
const bool USE_OP_POW      = {'true' if USE_OP_POW else 'false'};
const bool USE_OP_MOD      = {'true' if USE_OP_MOD else 'false'};
const bool USE_OP_SIN      = {'true' if USE_OP_SIN else 'false'};
const bool USE_OP_COS      = {'true' if USE_OP_COS else 'false'};
const bool USE_OP_LOG      = {'true' if USE_OP_LOG else 'false'};
const bool USE_OP_EXP      = {'true' if USE_OP_EXP else 'false'};
const bool USE_OP_FACT     = {'true' if USE_OP_FACT else 'false'};
const bool USE_OP_FLOOR    = {'true' if USE_OP_FLOOR else 'false'};
const bool USE_OP_GAMMA    = {'true' if USE_OP_GAMMA else 'false'};

// Tree Generation
const int MAX_TREE_DEPTH_INITIAL = 8;
const double TERMINAL_VS_VARIABLE_PROB = 0.75;
const double CONSTANT_MIN_VALUE = -10.0;
const double CONSTANT_MAX_VALUE = 10.0;
const int CONSTANT_INT_MIN_VALUE = -10;
const int CONSTANT_INT_MAX_VALUE = 10;
// Order: +, -, *, /, ^, %, s, c, l, e, !, _, g
const std::vector<double> OPERATOR_WEIGHTS = {{
    0.10 * (USE_OP_PLUS  ? 1.0 : 0.0),
    0.15 * (USE_OP_MINUS ? 1.0 : 0.0),
    0.10 * (USE_OP_MULT  ? 1.0 : 0.0),
    0.10 * (USE_OP_DIV   ? 1.0 : 0.0),
    0.05 * (USE_OP_POW   ? 1.0 : 0.0),
    0.01 * (USE_OP_MOD   ? 1.0 : 0.0),
    0.01 * (USE_OP_SIN   ? 1.0 : 0.0),
    0.01 * (USE_OP_COS   ? 1.0 : 0.0),
    0.15 * (USE_OP_LOG   ? 1.0 : 0.0),
    0.02 * (USE_OP_EXP   ? 1.0 : 0.0),
    0.05 * (USE_OP_FACT  ? 1.0 : 0.0),
    0.05 * (USE_OP_FLOOR ? 1.0 : 0.0),
    0.20 * (USE_OP_GAMMA ? 1.0 : 0.0)
}};

// Fitness & Other
const double COMPLEXITY_PENALTY_FACTOR = 0.05;
const bool USE_RMSE_FITNESS = true;
const double FITNESS_ORIGINAL_POWER = 1.3;
const double FITNESS_PRECISION_THRESHOLD = 0.001;
const double FITNESS_PRECISION_BONUS = 0.0001;
const double FITNESS_EQUALITY_TOLERANCE = 1e-9;
const double EXACT_SOLUTION_THRESHOLD = 1e-8;

// Weighted Fitness
const bool USE_WEIGHTED_FITNESS = {'true' if USE_WEIGHTED_FITNESS else 'false'};
const double WEIGHTED_FITNESS_EXPONENT = {WEIGHTED_FITNESS_EXPONENT};

// Advanced Features
const int STAGNATION_LIMIT_ISLAND = 50;
const int GLOBAL_STAGNATION_LIMIT = 5000;
const double STAGNATION_RANDOM_INJECT_PERCENT = 0.1;
const int PARAM_MUTATE_INTERVAL = 50;
const double PATTERN_RECORD_FITNESS_THRESHOLD = 10.0;
const int PATTERN_MEM_MIN_USES = 3;
const int PATTERN_INJECT_INTERVAL = 10;
const double PATTERN_INJECT_PERCENT = 0.05;
const size_t PARETO_MAX_FRONT_SIZE = 50;
const double SIMPLIFY_NEAR_ZERO_TOLERANCE = 1e-9;
const double SIMPLIFY_NEAR_ONE_TOLERANCE = 1e-9;
const int LOCAL_SEARCH_ATTEMPTS = 30;
const bool USE_SIMPLIFICATION = {'true' if USE_SIMPLIFICATION else 'false'};
const bool USE_ISLAND_CATACLYSM = {'true' if USE_ISLAND_CATACLYSM else 'false'};
const int PROGRESS_REPORT_INTERVAL = 100;
const bool FORCE_INTEGER_CONSTANTS = {'true' if FORCE_INTEGER_CONSTANTS else 'false'};

#include <random>
const double MUTATE_INSERT_CONST_PROB = 0.6;
const int MUTATE_INSERT_CONST_INT_MIN = 1;
const int MUTATE_INSERT_CONST_INT_MAX = 5;
const double MUTATE_INSERT_CONST_FLOAT_MIN = 0.5;
const double MUTATE_INSERT_CONST_FLOAT_MAX = 5.0;

std::mt19937& get_rng();
const double INF = std::numeric_limits<double>::infinity();

#endif // GLOBALS_H
"""

with open('src/Globals.h', 'w') as f:
    f.write(globals_content)
print("Globals.h updated with new parameters.")

In [ ]:
!cmake -B build -S . -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j $(nproc)

## Execution

In [ ]:
!./build/SymbolicRegressionGP